In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

# **1. DATA LOAD**

In [ ]:
df_book_tags = pd.read_csv("csv/book_tags.csv")
df_books = pd.read_csv("csv/books.csv")
df_ratings = pd.read_csv("csv/ratings.csv")
df_tags = pd.read_csv("csv/tags.csv")
df_to_read = pd.read_csv("csv/to_read.csv")

# **2. DATA CLEANING**

# **4.CONTENT BASED**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack # Para concatenar matrices dispersas y densas
import random

# **5. COLLABORATIVE**

El modelo colaborativo se basa en la similitud de usuarios o ítems según las interacciones históricas (por ejemplo, calificaciones, reseñas, etc.). En este caso, utilizaremos un enfoque basado en ítems, que recomienda libros similares a los que un usuario ya ha valorado positivamente.

In [ ]:
from scipy.sparse import csr_matrix

### **5.1 ENTENDIENDO LA DISTRIBUCIÓN DE REVIEWS POR USUARIO**

Accedemos al user_id único de df_ratings para generar los datos del usuario

In [ ]:
df_ratings['user_id'].value_counts().describe()

In [ ]:
df_ratings['user_id'].value_counts().mode()

In [ ]:
# Contar el número de reviews por usuario
user_review_counts = df_ratings['user_id'].value_counts()

# Crear el histograma
plt.figure(figsize=(10, 6))
sns.histplot(user_review_counts, bins=50, kde=False, color='blue')

# Etiquetas y título
plt.xlabel('Número de reviews por usuario')
plt.ylabel('Frecuencia')
plt.title('Distribución de reviews por usuario')
plt.yscale('log')  # Escala logarítmica para visualizar mejor la cola larga
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Mostrar el gráfico
plt.show()

Hay muchos usuarios con pocas reviews (pico en el mínimo), y la frecuencia decrece a medida que aumenta el número de reviews. 

Corte inferior:

El 25% de los usuarios tienen 3 reviews o menos.

Esto justifica un corte mínimo en 3, ya que los usuarios con menos reviews no aportan suficiente información para patrones útiles.

Corte superior:

El máximo es 200, y el 75% de los usuarios tienen 22 o menos reviews.

Esto sugiere que los usuarios con más de 50 reviews podrían ser considerados outliers

**5.1.1 Mediante el boxplot podemos visualizar los outliers**

In [ ]:
# Calcular rango intercuartílico (IQR) y límites para outliers
q1 = user_review_counts.quantile(0.25)
q3 = user_review_counts.quantile(0.75)
iqr = q3 - q1
lower_bound = max(3, q1 - 1.5 * iqr)  # Ajustamos mínimo a 3 reviews
upper_bound = q3 + 1.5 * iqr

# Crear boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x=user_review_counts, color='skyblue')

# Etiquetas y título
plt.title('Boxplot del número de reviews por usuario')
plt.xlabel('Número de reviews')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Mostrar el gráfico
plt.show()

# Imprimir límites sugeridos
print(f"Corte inferior sugerido (mínimo 3): {lower_bound}")
print(f"Corte superior sugerido: {upper_bound}")


### **5.2 GENERACION SINTETICA DE EL DF DE USERS**

In [ ]:
# Crear el dataset 'users' con user_id único
df_users = pd.DataFrame(df_ratings['user_id'].unique(), columns=['user_id'])

total_users_orig = len(df_users)

# Verificar el resultado
print(f"Número de usuarios únicos: {total_users_orig}")
df_users


##### **5.2.1 NUMERO DE REVIEWS QUE NOS INTERESAN**

En base a la distribución de reviews por usuario, podemos decir que el minimo de reviews por usuario que nos interesa es de 3, ya que menos de ese valor puede no darnos la información suficiente. Por otro lado, valores por encima de 50 reviews (los valores extremos o outliers) puede sesgar la información debido a la gran variedad de libros que puede llegar a leer y valorar.

In [ ]:
# Filtrar usuarios con reviews en el rango definido
filtered_users = user_review_counts[
    (user_review_counts >= 3) & (user_review_counts <= upper_bound)
].index

# Aplicar el filtro a df_ratings y df_users
df_ratings = df_ratings[df_ratings['user_id'].isin(filtered_users)]
df_users = df_users[df_users['user_id'].isin(filtered_users)]

# Verificar resultados
print(f"Número de usuarios tras el filtrado: {len(df_users)}")
print(f"Número de reviews tras el filtrado: {len(df_ratings)}")

total_users_after = len(df_users[df_users['user_id'].isin(filtered_users)])

df_users


In [ ]:
percentage_removed = ((total_users_orig - total_users_after) / total_users_orig) * 100

# Porcentaje de usuarios mantenidos
percentage_kept = (total_users_after / total_users_orig) * 100

# Imprimir resultados
print(f"Total usuarios antes del filtrado: {total_users_orig}")
print(f"Total usuarios después del filtrado: {total_users_after}")
print(f"Porcentaje de usuarios eliminados: {percentage_removed:.2f}%")
print(f"Porcentaje de usuarios mantenidos: {percentage_kept:.2f}%")

**Tan solo eliminamos el 25% de los datos, quedandonos con 40.000 usuarios, un número todavia significativo**

### **5.3 ASIGNACIÓN DE VALORES A LOS USUARIOS FILTRADOS**

Para la generación de datos sintéticos hemos escogido variables como **age** (basada en la piramide de la población mundial) que va de 16 a 60 años, **gender**(Male o Female) distribuyendo su importancia de forma equitativa, **education** teniendo en cuenta que para cierta edad no has podido acceder a cierto nivel educativo y **country** asignando mas peso a paises occidentales que a los orientales/asiáticos

In [ ]:
# Semilla para reproducibilidad
np.random.seed(42)

# Generar columna 'age' (16-70 años, ajustado según la pirámide poblacional)
age_distribution = np.array([0.25, 0.35, 0.25, 0.10, 0.05])
age_distribution /= age_distribution.sum() # Normalizar por si acaso

age_ranges = [(16, 25), (25, 35), (35, 50), (50, 65), (65, 71)]
ages = []
num_users = len(df_users)

for idx, (low, high) in enumerate(age_ranges):
    num_in_range = int(num_users * age_distribution[idx])
    ages.extend(np.random.randint(low, high, size=num_in_range))

# Ajustar la longitud si hay una pequeña diferencia debido al redondeo
while len(ages) < num_users:
    ages.append(np.random.randint(age_ranges[-1][0], age_ranges[-1][1])) # Añadir edad del último rango
while len(ages) > num_users:
    ages.pop() # Eliminar edad

np.random.shuffle(ages)
df_users['age'] = ages

# Generar columna 'gender' (50% M/F)
df_users['gender'] = np.random.choice(['M', 'F'], size=num_users)

# Generar columna 'education' (niveles educativos ajustados)
def assign_education(age):
    if age < 18:
        return 'School'
    elif age < 22:
        return np.random.choice(['High School', 'Vocational Training'], p=[0.7, 0.3])
    elif age < 30:
        return np.random.choice(["High School", "Bachelor's", "Vocational Training"], p=[0.3, 0.5, 0.2])
    elif age < 40:
        return np.random.choice(["Bachelor's", "Master's", "Vocational Training"], p=[0.4, 0.3, 0.3])
    else:
        return np.random.choice(["Bachelor's", "Master's", "PhD", "Vocational Training"], p=[0.4, 0.2, 0.1, 0.3])

df_users['education'] = df_users['age'].apply(assign_education)

# Generar columna 'country' (muchos más países, prioridades ajustadas)
countries = [
    'Spain', 'France', 'Germany', 'Italy', 'UK', 'Portugal', 'Belgium', 'Netherlands',
    'Switzerland', 'Austria', 'Sweden', 'Norway', 'Denmark', 'Ireland', 'Finland',
    'Greece', 'Poland', 'Czech Republic', 'Hungary', 'Romania', 'Ukraine', 'Russia',
    'Turkey', 'Croatia', 'Serbia', 'Bulgaria', 'Lithuania', 'Latvia', 'Estonia',
    'Slovakia', 'Slovenia', 'Bosnia and Herzegovina', 'Albania', 'North Macedonia',
    'Montenegro', 'Iceland', 'Luxembourg', 'Malta', 'Cyprus',

    'USA', 'Canada', 'Mexico', 'Brazil', 'Argentina', 'Colombia', 'Chile', 'Peru',
    'Venezuela', 'Ecuador', 'Bolivia', 'Paraguay', 'Uruguay', 'Cuba', 'Dominican Republic',
    'Puerto Rico', 'Guatemala', 'Honduras', 'El Salvador', 'Nicaragua', 'Costa Rica',
    'Panama', 'Jamaica', 'Haiti', 'Trinidad and Tobago', 'Bahamas', 'Barbados',
    'Belize', 'Guyana', 'Suriname',

    'China', 'Japan', 'South Korea', 'India', 'Indonesia', 'Pakistan', 'Bangladesh',
    'Vietnam', 'Thailand', 'Philippines', 'Saudi Arabia', 'Iran', 'Iraq', 'Israel',
    'Singapore', 'Malaysia', 'Taiwan', 'Hong Kong', 'Sri Lanka', 'Nepal', 'Myanmar',

    'South Africa', 'Egypt', 'Nigeria', 'Kenya', 'Ethiopia', 'Morocco', 'Algeria',
    'Ghana', 'Ivory Coast', 'Chad', # Added 'Oops' as an example, user can replace or remove

    'Australia', 'New Zealand'
]

# Pesos aproximados para priorizar Europa y América
weights = np.array([
    10, 10, 10, 10, 10, 8, 8, 8, 7, 7, 6, 6, 6, 6, 5, 5, 5, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2,
    2, 2, 1, 1, 1, 1, 3, 3, 2, 2, # Europe weights

    15, 10, 10, 9, 9, 7, 7, 6, 5, 5, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 1, 2, 2, 2,
    1, 1, 1, # Americas weights

    5, 5, 5, 5, 3, 2, 2, 3, 3, 3, 3, 2, 1, 3, 3, 3, 3, 3, 2, 1, 1, # Asia weights

    4, 4, 3, 2, 1, 2, 1, 2, 1, 1, # Africa weights

    6, 4 # Oceania weights
])

# Asegurarse de que el número de pesos coincida con el número de países
if len(countries) != len(weights):
    raise ValueError("El número de países y pesos no coincide.")

# Normalizar los pesos para obtener probabilidades
country_distribution = weights / weights.sum()

df_users['country'] = np.random.choice(countries, size=num_users, p=country_distribution)

# Verificar los datos generados
df_users.head()

Le vamos a asignar también un **reading_preference** a cada usuario, basandonos el género que más ha valorado/leido. Nos devolverá el género que más ha valorado/leido, y en caso que no haya un género que más haya leído, se le asignará uno aleatorio

In [ ]:
# Obtener géneros únicos de df_books
unique_genres = df_combined_books['genre'].unique()

# Función para determinar el género preferido
def assign_reading_preference(user_id, ratings_df, books_df):
    # Filtrar libros leídos por el usuario
    user_books = ratings_df[ratings_df['user_id'] == user_id]['book_id']
    user_genres = books_df[books_df['book_id'].isin(user_books)]['genre']
    
    # Si tiene géneros, devolver el más frecuente
    if not user_genres.empty:
        return user_genres.mode().iloc[0]  # Género más frecuente
    # Si no tiene géneros, asignar uno aleatorio
    else:
        return np.random.choice(unique_genres)

# Asignar la preferencia de lectura a cada usuario
df_users['reading_preference'] = df_users['user_id'].apply(
    lambda uid: assign_reading_preference(uid, df_ratings, df_combined_books)
)

dicebear_base_url = "https://api.dicebear.com/7.x/identicon/svg?seed="
df_users['avatar_url'] = dicebear_base_url + df_users['user_id'].astype(str)

# Verificar datos generados
df_users


### **5.4 USER-BASED**

El objetivo es encontrar usuarios con gustos similares. Si dos usuarios califican libros de manera parecida, se asume que tienen gustos similares. Recomendamos libros que un usuario similar haya calificado alto, pero que el usuario objetivo no haya leído aún.

---------------------------------------------------------------------------------------------------------------------------------------------

**Antes de nada, veamos si los book_id de df_ratings se encuentran o corresponden con los book_id de nuestra librería porque si no, obtendremos libros desconocidos**

In [ ]:
# 1. Obtener el conjunto de book_id que existen en nuestro catálogo
valid_book_ids_in_catalog = set(df_combined_books['book_id'])

# 2. Obtener el conjunto de book_id que aparecen en las calificaciones
book_ids_in_ratings = set(df_ratings['book_id'])

# 3. Identificar los book_id en ratings que NO están en el catálogo
unknown_book_ids_in_ratings = book_ids_in_ratings - valid_book_ids_in_catalog

# 4. Contar cuántos book_id únicos son desconocidos
num_unique_unknown_ids = len(unknown_book_ids_in_ratings)

print(f"Total de book_id únicos en df_ratings: {len(book_ids_in_ratings)}")
print(f"Total de book_id únicos conocidos en df_combined_books: {len(valid_book_ids_in_catalog)}")
print(f"Total de book_id únicos desconocidos encontrados en df_ratings (no en el catálogo): {num_unique_unknown_ids}")


# 5. Contar cuántas filas de df_ratings están afectadas por estos IDs desconocidos
#    Esto es el número de calificaciones que se refieren a libros desconocidos
rows_with_unknown_id = df_ratings[df_ratings['book_id'].isin(unknown_book_ids_in_ratings)]
num_rows_with_unknown_id = len(rows_with_unknown_id)
total_ratings = len(df_ratings)

# 6. Calcular el porcentaje de filas (calificaciones) afectadas
percentage_affected_rows = (num_rows_with_unknown_id / total_ratings) * 100 if total_ratings > 0 else 0

print(f"Número de filas (calificaciones) afectadas por book_id desconocidos: {num_rows_with_unknown_id}")
print(f"Porcentaje de filas en df_ratings con book_id desconocidos: {percentage_affected_rows:.2f}%")

El 91.90% de los book_id de df_rating no coinciden con los book_id de mi biblioteca o no están, básicamente. Problema. Esto quiere decir que de cada 10 recomendaciones 9 serán libros desconocidos. Para subsanar esto de una manera "lógica" vamos a asignar o cambiar un book_id de la biblioteca (que existe) con el book_id de los ratings que no se encuentra en la biblioteca, basándonos en el reading_preference del user, es decir, del tipo de género que más lee cada usuario.

In [ ]:
# --- PASO 1: Identificar book_id válidos y desconocidos ---
valid_book_ids = set(df_combined_books['book_id'])
unknown_book_ids = set(df_ratings['book_id']) - valid_book_ids
print(f"Total de book_id desconocidos encontrados en df_ratings: {len(unknown_book_ids)}")

# --- CALCULAR PORCENTAJE DE IMPUTACIÓN ---
rows_with_unknown_id = df_ratings[df_ratings['book_id'].isin(unknown_book_ids)]
num_rows_with_unknown_id = len(rows_with_unknown_id)
total_ratings = len(df_ratings)
percentage_affected_rows = (num_rows_with_unknown_id / total_ratings) * 100 if total_ratings > 0 else 0

print(f"Número de filas (calificaciones) afectadas por book_id desconocidos: {num_rows_with_unknown_id}")
print(f"Porcentaje de filas en df_ratings con book_id desconocidos: {percentage_affected_rows:.2f}%")


# --- OPTIMIZACIÓN 1: Crear mapa de preferencia de usuario ---
print("\nCreando mapa de preferencias de usuario para búsqueda rápida...")
user_preference_map = df_users.set_index('user_id')['reading_preference']


# --- OPTIMIZACIÓN 2: Pre-filtrar libros por género/preferencia ---
# Vamos a encontrar todos los book_id que coinciden con cada preferencia ÚNICA, UNA SOLA VEZ.
print("Pre-filtrando y mapeando libros por preferencias de género únicas...")

# Obtener las preferencias únicas y válidas de todos los usuarios
# Usamos dropna().unique() para obtener solo valores no nulos y únicos
# Filtramos para asegurar que son strings no vacíos y limpiamos espacios
unique_preferences = user_preference_map.dropna().unique()
unique_preferences = [p.strip() for p in unique_preferences if isinstance(p, str) and p.strip()]

# Crear un diccionario para almacenar la lista de book_id para cada preferencia
genre_book_ids_map = {}

# Asegurarse de que la columna 'genre' en df_combined_books es de tipo string para .str
df_combined_books['genre'] = df_combined_books['genre'].astype(str)

for preference in unique_preferences:
    try:
        # Filtramos df_combined_books UNA SOLA VEZ para esta preferencia
        matching_books = df_combined_books[
            df_combined_books['genre'].str.contains(
                preference, na=False, case=False
            )
        ]
        # Almacenar la lista de book_id que coinciden con esta preferencia
        if not matching_books.empty:
            genre_book_ids_map[preference] = matching_books['book_id'].tolist()
        else:
            genre_book_ids_map[preference] = [] # Si no hay coincidencias, lista vacía

    except Exception as e:
        print(f"Advertencia durante pre-filtrado de '{preference}': {e}")
        genre_book_ids_map[preference] = [] # En caso de error, lista vacía

print(f"Pre-filtrado completado para {len(unique_preferences)} preferencias únicas.")


# --- PASO 2: Definir la función de asignación aún más optimizada ---

# La función ahora usa el mapa pre-calculado (genre_book_ids_map)
def assign_book_id_by_preference_highly_optimized(row, valid_book_ids, unknown_book_ids, user_preference_map, genre_book_ids_map):
    """
    Asigna un book_id válido basado en la preferencia de lectura del usuario
    (usando mapas optimizados y pre-filtrado) si el book_id original es desconocido.
    Si no hay preferencia, usuario desconocido, o no hay género coincidente,
    asigna un book_id válido aleatorio.
    """
    current_book_id = row['book_id']

    # Si el book_id actual ya es conocido, no hacemos nada
    # La verificación en un set es rápida
    if current_book_id in valid_book_ids:
        return current_book_id

    # Si el book_id actual es desconocido
    user_id = row['user_id']

    # --- Buscar la preferencia del usuario (Optimizado 1 con mapa) ---
    # Usa .get() en la Serie user_preference_map, muy rápido
    preferred_genre = user_preference_map.get(user_id)

    # Asegurarse de que preferred_genre es una cadena válida y no está vacía
    if not pd.notna(preferred_genre) or not isinstance(preferred_genre, str) or preferred_genre.strip() == "":
         preferred_genre = None # Tratar como si no hubiera una preferencia válida


    # --- Buscar libros válidos que coincidan con el género preferido (Optimizado 2 con pre-filtrado) ---
    chosen_book_id = None
    # Si hay una preferencia válida y esa preferencia está en nuestro mapa de pre-filtrado
    if preferred_genre and preferred_genre.strip() in genre_book_ids_map:
        # Obtener la lista de book_id pre-calculada para esta preferencia
        matching_book_ids_list = genre_book_ids_map.get(preferred_genre.strip())

        # Si la lista no está vacía, elegimos un book_id aleatorio de ella
        if matching_book_ids_list:
             chosen_book_id = random.choice(matching_book_ids_list)

    # --- Asignar el nuevo book_id (Fallback) ---
    # Si no se asignó un book_id basado en la preferencia (porque no había preferencia,
    # no se encontró en el mapa, o la lista estaba vacía), asignamos uno aleatorio de todos los válidos.
    if chosen_book_id is None:
         if valid_book_ids:
              chosen_book_id = random.choice(list(valid_book_ids))
         else:
              # Esto no debería pasar, pero lo manejamos
              # print("Error: No hay book_id válidos generales disponibles para asignar.")
              return None # Devolver None o algún indicador de error

    return chosen_book_id


# --- PASO 3: Aplicar la función optimizada a las filas de df_ratings ---
# Creamos una copia para no modificar el DataFrame original directamente
df_ratings_modified = df_ratings.copy()

print(f"Aplicando corrección de book_id (altamente optimizado) a {num_rows_with_unknown_id} filas...")

# Aplicar la función fila por fila (axis=1)
# Pasamos todos los argumentos necesarios, incluyendo los mapas y el mapa de pre-filtrado
df_ratings_modified['book_id'] = df_ratings_modified.apply(
    lambda row: assign_book_id_by_preference_highly_optimized(
        row, valid_book_ids, unknown_book_ids, user_preference_map, genre_book_ids_map
    ),
    axis=1
)

print("Proceso de corrección de book_id completado.")


# --- PASO 4: Verificar el resultado ---
remaining_unknown_ids = set(df_ratings_modified['book_id']) - valid_book_ids
if not remaining_unknown_ids:
    print("\nVerificación: Todos los book_id ahora son válidos en df_ratings_modified.")
else:
    print(f"\nVerificación: Aún quedan {len(remaining_unknown_ids)} book_id desconocidos en df_ratings_modified.")

# Opcional: Verificar si algún valor se asignó como None en caso de error extremo
if (df_ratings_modified['book_id'].isna()).any():
     print("Advertencia: Se asignaron valores nulos a algunos book_id durante la corrección.")

# df_ratings_modified ahora contiene los book_id corregidos. Puedes guardarlo si quieres:
# df_ratings_modified.to_csv('df_ratings_corregido.csv', index=False)

In [ ]:
book_id_to_search = random.choice(df_ratings_modified["book_id"].unique())
print("book_id to search in library:", book_id_to_search)

print(df_combined_books[df_combined_books["book_id"]==book_id_to_search][["title", "book_id"]])


In [ ]:
print(df_ratings.shape)
print(df_ratings_modified.shape)

In [ ]:
# Verificar si todos los book_id de df_ratings están en df_combined_books
ratings_book_ids = set(df_ratings_modified['book_id'])
books_combined_ids = set(df_combined_books['book_id'])

# Identificar book_id que no coinciden
missing_book_ids = ratings_book_ids - books_combined_ids

print(f"Total de book_id en df_ratings: {len(ratings_book_ids)}")
print(f"Total de book_id en df_combined_books: {len(books_combined_ids)}")
print(f"Book_id faltantes en df_combined_books: {len(missing_book_ids)}")
print(f"Ejemplo de book_id faltantes: {list(missing_book_ids)[:10]}")

In [ ]:
df_ratings_modified

**Ya existen todos los book_id de la libreria en el rating de los usuarios**

--------------

Para que un sistema de recomendación colabore entre usuarios, necesita ver esta información de una manera que le permita comparar fácilmente qué libros ha leído cada usuario y qué libros no ha leído. Para eso hacemos la pivot-table, donde:  
1. Cada fila representa a un usuario único.
  
2. Cada columna representa un libro único de tu catálogo.
  
3. Dentro de cada celda de la cuadrícula, verás la calificación que ese usuario le dio a ese libro.

In [ ]:
# Crear la matriz User-Item
user_item_matrix = df_ratings_modified.pivot_table(
    index='user_id',
    columns='book_id',
    values='rating'
).fillna(0)

# Verificar la forma de la matriz
print(f"Forma de la matriz User-Item: {user_item_matrix.shape}")
user_item_matrix


In [ ]:
print(len(df_ratings_modified["book_id"].value_counts()))

En lugar de generar una matriz de similitud de 40,304 x 40,304, usamos KNN para encontrar los vecinos más cercanos. Además usamos una Compressed Sparse Row matrix (csr_matrix) para comprimir los datos sin perder información para acelerar el proceso del grid search.

In [ ]:
user_item_csr = csr_matrix(user_item_matrix.values)
user_item_csr

**¿Por qué no train/test split?** 
  
En este tipo particular de sistema de recomendación basado en filtrado colaborativo de usuario-a-usuario (User-Based Collaborative Filtering), el modelo NearestNeighbors no está aprendiendo a predecir una etiqueta o un valor en el sentido de aprendizaje supervisado. El modelo NearestNeighbors esencialmente está construyendo una estructura de datos o preparando un algoritmo para encontrar rápidamente los puntos (usuarios) en la matriz que son más cercanos a un punto de consulta dado.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors

param_grid = {
    'n_neighbors': [5, 10, 20, 50],
    'metric': ['cosine', 'euclidean', 'manhattan'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

knn_model = NearestNeighbors(n_jobs=-1)

grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1) 
grid_search.fit(user_item_csr)

# Imprimir los mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)

Algoritmo KNN: Busca filas (usuarios) en la matriz que sean "cercanas" o similares a la fila del usuario objetivo basándose en un métrica (coseno), con los mejores parametros del grid_search

---------------------

El método knn_best.fit(user_item_csr) le dice al modelo NearestNeighbors "Aquí tienes el espacio de datos en el que vas a buscar vecinos". Preparara la estructura de datos y el algoritmo para una búsqueda eficiente de vecinos dentro de tu conjunto completo de usuarios.

In [ ]:
knn_best = NearestNeighbors(metric='cosine', n_jobs=-1,n_neighbors=5,algorithm='auto') 
knn_best.fit(user_item_csr)

Identifica Vecinos Similares: Resultado del KNN.

In [ ]:
# Asegurarse de que la matriz no está vacía
if not user_item_matrix.index.empty:
    # 1. Seleccionar un user_id ORIGINAL aleatorio de la matriz (del índice)
    user_id_original = random.choice(user_item_matrix.index)

    # 2. Encontrar el índice POSICIONAL de este user_id original en la matriz CSR
    #    (Necesitamos el índice posicional para knn_best.kneighbors)
    user_matrix_index = user_item_matrix.index.get_loc(user_id_original)

    top_k = 10  # Número de usuarios más similares a mostrar (excluyendo el usuario mismo)

    # 3. Encontrar los vecinos más cercanos usando el índice POSICIONAL en la matriz CSR
    #    Pedimos top_k + 1 para incluir al propio usuario
    distances, indices = knn_best.kneighbors(user_item_csr[user_matrix_index], n_neighbors=top_k + 1)

    # 4. Ignorar el primer resultado (el usuario mismo)
    #    Los índices devueltos aquí son índices POSICIONALES en la matriz CSR
    similar_users_matrix_indices = indices[0][1:]
    similar_distances = distances[0][1:]

    # 5. Mapear los índices POSICIONALES de los usuarios similares de vuelta a sus user_id ORIGINALES
    #    Usamos el índice del DataFrame user_item_matrix para hacer este mapeo
    similar_users_original_ids = user_item_matrix.index[similar_users_matrix_indices]


    # 6. Crear DataFrame de resultados con los user_id ORIGINALES
    similar_users_df = pd.DataFrame({
        'Similar_User_ID_Original': similar_users_original_ids, # Usamos los IDs originales aquí
        'Distance': similar_distances
    })

    # Mostrar resultados
    # Nos referimos al usuario seleccionado por su user_id ORIGINAL
    print(f"Usuarios más similares al usuario {user_id_original} (menor distancia = mayor similitud):")
    print(similar_users_df)

else:
    print("La matriz User-Item está vacía. No se pueden encontrar vecinos.")

In [ ]:

rows, cols = user_item_csr.nonzero()

plt.figure(figsize=(10, 8))
plt.scatter(cols, rows, s=1, alpha=0.5) # s es el tamaño del punto
# plt.title('Dispersión de la Matriz User-Item (elementos no cero)')
# plt.xlabel('Índice de Libro')
# plt.ylabel('Índice de Usuario')
# plt.grid(False) # Desactivar cuadrícula para una mejor visualización de puntos
# plt.show()


sns.heatmap(user_item_matrix.iloc[:99, :99], cmap='viridis')
plt.title('Heatmap de una porción de la Matriz User-Item')
plt.show()

In [ ]:
ratings_per_user = (user_item_matrix > 0).sum(axis=1) # Contar no-ceros por fila (usuario)
plt.figure(figsize=(10, 6))
ratings_per_user.hist(bins=50) # O usa sns.histplot() para más opciones
plt.title('Distribución del Número de Libros Calificados por Usuario')
plt.xlabel('Número de Libros Calificados')
plt.ylabel('Número de Usuarios')
# plt.yscale('log') # Escala logarítmica en Y si hay muchos usuarios con pocas calificaciones
plt.show()

In [ ]:
sample_size = 2 # Número de usuarios de muestra
all_neighbor_distances = []

# Seleccionar índices de muestra aleatoriamente
if user_item_matrix.index.empty:
    print("La matriz User-Item está vacía.")
else:
    sample_user_indices = random.sample(range(user_item_csr.shape[0]), min(sample_size, user_item_csr.shape[0]))

    for user_idx in sample_user_indices:
            # Encontrar top k+1 vecinos (incluyendo el usuario)
        distances, indices = knn_best.kneighbors(user_item_csr[user_idx], n_neighbors=knn_best.n_neighbors + 1)
            # Añadir distancias (excluyendo la del propio usuario, que es 0)
        all_neighbor_distances.extend(distances[0][1:])

        # Convertir a similitudes (si usaste métrica de distancia)
        # Si usaste 'cosine', la distancia es 1 - similitud coseno.
        # Si usaste 'euclidean' o 'manhattan', la distancia es la métrica.
        # Para plotear similitud, si la métrica es distancia, puedes usar 1 / (1 + distancia) o 1 - distancia si la métrica es acotada como coseno.
        # Si metric='cosine', distance is 1 - cosine_similarity
        if knn_best.metric == 'cosine':
             all_neighbor_similarities = [1 - d for d in all_neighbor_distances]
             plt.figure(figsize=(10, 6))
             plt.hist(all_neighbor_similarities, bins=30)
             plt.title('Distribución de la Similitud Coseno de los Vecinos')
             plt.xlabel('Similitud Coseno')
             plt.ylabel('Frecuencia')
             plt.show()
        else:
             # Si usaste otras métricas (Euclidiana, Manhattan), plotear la distribución de distancias
             plt.figure(figsize=(10, 6))
             plt.hist(all_neighbor_distances, bins=30)
             plt.title(f'Distribución de la Distancia ({knn_best.metric}) de los Vecinos')
             plt.xlabel('Distancia')
             plt.ylabel('Frecuencia')
             plt.show()

### **5.4.1 User-Based Recommendation Results**

Se identifican todos los libros calificados por los vecinos.
  
Se eliminan de la lista los libros que el usuario ya ha valorado.
  
Para los libros restantes, se suma la similitud de los vecinos que lo calificaron.
  
Se ordenan los libros de mayor a menor relevancia.
  
Se seleccionan los N  libros con mayor relevancia.

In [ ]:
import pandas as pd
import random
from IPython.display import HTML, display


# Asegurarse de que book_id es entero y crear diccionarios de mapeo
df_combined_books['book_id'] = df_combined_books['book_id'].astype(int)
book_id_to_title = dict(zip(df_combined_books['book_id'], df_combined_books['title']))
book_id_to_image_url = dict(zip(df_combined_books['book_id'], df_combined_books['image_url']))
# Asegúrate de que la columna de autor se llama 'authors' en df_combined_books
if 'authors' not in df_combined_books.columns:
     print("Advertencia: La columna 'authors' no se encontró en df_combined_books. Usando 'Autor desconocido'.")


# Seleccionar un usuario válido aleatoriamente basado en los índices existentes
# Asegúrate de que user_item_matrix tiene un índice no vacío
if user_item_matrix.index.empty:
    print("Error: La matriz User-Item está vacía. No se puede seleccionar un usuario.")
else:
    # Selecciona un user_id ORIGINAL aleatorio de la matriz (del índice)
    user_id = random.choice(user_item_matrix.index)

    # --- INICIO: SECCIÓN PARA MOSTRAR EL AVATAR Y DATOS DEL USUARIO OBJETIVO ---
    # Buscar la información del usuario seleccionado en df_users
    user_row = df_users[df_users['user_id'] == user_id]

    if not user_row.empty:
        # Extraer datos de la primera (y única) fila encontrada
        user_avatar_url = user_row.iloc[0].get('avatar_url', 'placeholder_avatar_url') # Fallback URL
        user_username = user_row.iloc[0].get('username', f'Usuario {user_id}')
        user_age = user_row.iloc[0].get('age', 'Edad desconocida')
        user_gender = user_row.iloc[0].get('gender', 'Género desconocido')
        user_country = user_row.iloc[0].get('country', 'País desconocido')
        user_reading_pref = user_row.iloc[0].get('reading_preference', 'Preferencias desconocidas')

        # Generar el HTML para mostrar el avatar y la información del usuario
        user_html = f"""
        <div style="display: flex; align-items: center; margin-bottom: 20px; border-bottom: 1px solid #eee; padding-bottom: 15px;">
            <div style="margin-right: 20px;">
                <img src="{user_avatar_url}" alt="Avatar de {user_username}" width="100" style="border-radius: 50%; border: 2px solid #ccc; object-fit: cover;">
            </div>
            <div>
                <h3 style="margin: 0 0 5px 0; color: #eee;">Perfil del Usuario Seleccionado: <span style="color: #007bff; font-weight: bold;">{user_username}</span> (ID: {user_id})</h3>
                <p style="margin: 0; font-size: 14px; color: #eee;">Edad: {user_age}</p>
                <p style="margin: 0; font-size: 14px; color: #eee;">Género: {user_gender}</p>
                <p style="margin: 0; font-size: 14px; color: #eee;">País: {user_country}</p>
                <p style="margin: 0; font-size: 14px; color: #eee;">Preferencia de lectura: {user_reading_pref}</p>
            </div>
        </div>
        """
        # Mostrar el HTML
        display(HTML(user_html))

    else:
        print(f"Error: No se encontró información del usuario objetivo {user_id} en df_users.")


    # --- FIN: SECCIÓN PARA MOSTRAR EL AVATAR Y DATOS DEL USUARIO OBJETIVO ---


    # Encontrar vecinos más cercanos para el usuario seleccionado
    # Asegurarse de que el user_id seleccionado existe en el índice antes de obtener su localización
    if user_id not in user_item_matrix.index:
        print(f"Error: user_id {user_id} no encontrado en user_item_matrix index.")
    else:
        user_index = user_item_matrix.index.get_loc(user_id)
        # Asegurarse de que user_item_csr tiene suficientes filas
        if user_index < user_item_csr.shape[0]:
             # Definimos cuántos vecinos queremos encontrar (y potencialmente mostrar)
             # top_k_neighbors_find = 10 # Puedes usar esto si quieres encontrar más de los que muestras
             top_k_neighbors_display = 4 # <-- Número de vecinos SIMILARES a mostrar en el display
             top_k_neighbors_find = max(10, top_k_neighbors_display + 1) # Asegúrate de encontrar al menos suficientes


             # Encontrar los vecinos más cercanos usando el índice POSICIONAL en la matriz CSR
             # Pedimos top_k_neighbors_find + 1 para incluir al propio usuario
             distances, indices = knn_best.kneighbors(user_item_csr[user_index], n_neighbors=top_k_neighbors_find + 1)


             # Ignorar el primer resultado (el usuario mismo)
             # Estos indices son POSICIONALES en la matriz CSR
             # similar_users_matrix_indices = indices[0][1:] # Esto toma todos los encontrados menos el primero
             # similar_distances = distances[0][1:] # Lo mismo para las distancias

             # Tomar SOLO los primeros N vecinos para el cálculo de recomendaciones y display
             similar_users_matrix_indices = indices[0][1 : top_k_neighbors_display + 1] # Indices posicionales de los top N a mostrar
             similar_distances = distances[0][1 : top_k_neighbors_display + 1] # Distancias de los top N a mostrar


             # Mapear los índices POSICIONALES de los usuarios similares de vuelta a sus user_id REALES
             similar_users_original_ids = user_item_matrix.index[similar_users_matrix_indices]


             # Crear DataFrame de usuarios similares y similitudes (solo para los que vamos a usar/mostrar)
             similarity_df = pd.DataFrame({
                 'user_id': similar_users_original_ids,
                 'similarity': 1 - similar_distances # Convertir distancia a similitud (1 - distancia)
             })


             # --- INICIO: SECCIÓN AÑADIDA PARA MOSTRAR USUARIOS SIMILARES ---
             # Recuperar la información de estos usuarios similares desde df_users
             # Usamos isin para filtrar por la lista de IDs de vecinos
             top_similar_users_info = df_users[df_users['user_id'].isin(similar_users_original_ids)].set_index('user_id')

             if not top_similar_users_info.empty:
                 similar_users_html = "<h3>Usuarios Similares:</h3>"
                 similar_users_html += "<div style='display: flex; flex-wrap: wrap; gap: 10px; margin-bottom: 20px;'>"

                 # Itera a través de los IDs de los top vecinos para mantener el orden por similitud
                 for sim_user_id in similar_users_original_ids: # Iteramos sobre los IDs originales de los vecinos top N
                     # Usa .get() en el DataFrame indexado para buscar la info del vecino
                     sim_user_info = top_similar_users_info.loc[sim_user_id] if sim_user_id in top_similar_users_info.index else None

                     if sim_user_info is not None:
                         # Obtener avatar y nombre del vecino
                         sim_user_avatar_url = sim_user_info.get('avatar_url', 'placeholder_avatar_url') # Fallback URL
                         sim_user_username = sim_user_info.get('username', f'Usuario {sim_user_id}')

                         similar_users_html += f"""
                         <div style="text-align: center; width: 80px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;">
                             <img src="{sim_user_avatar_url}" alt="Avatar de {sim_user_username}" width="50" style="border-radius: 50%; border: 1px solid #ccc; object-fit: cover; height: 50px;"><br>
                             <span style="font-size: 10px;" title="{sim_user_username}">{sim_user_username}</span>
                         </div>
                         """
                     # Manejar el caso si la info del vecino no se encuentra en df_users (menos probable)
                     # else:
                         # similar_users_html += f"""
                         # <div style="text-align: center; width: 80px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap; background-color: #f0f0f0;">
                         #     <div style="width: 50px; height: 50px; border-radius: 50%; border: 1px solid #ccc; margin: 0 auto; display: flex; align-items: center; justify-content: center; font-size: 8px;">Info Missing</div><br>
                         #     <span style="font-size: 10px;">ID: {sim_user_id}</span>
                         # </div>
                         # """


                 similar_users_html += "</div>"
                 display(HTML(similar_users_html))
             else:
                  print("\nNo se encontraron usuarios similares para mostrar.") # Debería encontrar si top_k > 0 y hay vecinos

             # --- FIN: SECCIÓN AÑADIDA PARA MOSTRAR USUARIOS SIMILARES ---


             # Obtener libros calificados por el usuario objetivo
             # Asegurarse de que el user_id objetivo existe en user_item_matrix index
             if user_id in user_item_matrix.index:
                  user_rated_books = set(user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index)
             else:
                  user_rated_books = set() # Si el usuario no está en la matriz, no tiene libros calificados aquí


             # Libros calificados por usuarios similares
             # NOTA: Aquí usamos la similarity_df que solo contiene los top_k_neighbors_display
             # Si quieres usar más vecinos para CALCULAR las recomendaciones de los que MUESTRAS,
             # necesitarías crear una similarity_df separada con más vecinos encontrados inicialmente.
             recommended_books = {}

             for _, row in similarity_df.iterrows(): # Itera sobre los top N vecinos que mostraste
                 sim_user = row['user_id']
                 sim_score = row['similarity']

                 # Libros calificados por el usuario similar
                 # Asegurarse de que sim_user existe en user_item_matrix index
                 if sim_user in user_item_matrix.index:
                      # Obtener solo los libros calificados positivamente (> 0)
                      sim_user_rated_books = set(user_item_matrix.loc[sim_user][user_item_matrix.loc[sim_user] > 0].index)

                      # Calcular relevancia basada en similitud
                      for book in sim_user_rated_books:
                          if book not in user_rated_books: # Evitar libros ya calificados por el usuario objetivo
                              if book not in recommended_books:
                                  recommended_books[book] = 0
                              recommended_books[book] += sim_score # Sumar la similitud del vecino


             # Asegurarnos de filtrar los libros recomendados que no están en df_combined_books
             recommended_books_filtered = {
                 book_id: score
                 for book_id, score in recommended_books.items()
                 if book_id in book_id_to_title # Esto implícitamente comprueba si está en la lista de book_id de df_combined_books
             }


             # Ordenar libros por relevancia
             sorted_recommended_books = sorted(recommended_books_filtered.items(), key=lambda x: x[1], reverse=True)

             # Mostrar los 10 libros más relevantes con títulos
             top_books = sorted_recommended_books[:10]

             # --- NEW: Displaying Recommended Books with Covers ---
             # Puedes añadir el nombre del usuario objetivo aquí si lo tienes disponible
             print(f"\nLibros recomendados para el usuario {user_id} ({user_username}):")


             if not top_books:
                 print("No se encontraron recomendaciones de libros.")
             else:
                 # Use HTML and flexbox for a nice layout of book covers
                 html_content = "<div style='display: flex; flex-wrap: wrap; gap: 15px;'>"

                 for book_id, score in top_books:
                     book_id_int = int(book_id) # Ensure integer type for dictionary lookup

                     # --- Retrieve full book details from df_combined_books ---
                     book_details = df_combined_books[df_combined_books['book_id'] == book_id_int]

                     if not book_details.empty:
                         details = book_details.iloc[0]

                         book_title = details.get('title', "Título desconocido")
                         image_url = details.get('image_url')
                         author = details.get('authors', "Autor desconocido") # Usamos 'authors' como encontraste
                         genre = details.get('genre', "Género desconocido")
                         pages = details.get('pages', "Páginas desconocidas")

                         # Convert pages to int if it's not None/NaN and looks like a number
                         try:
                             if pd.notna(pages):
                                 pages_str = f"{int(pages)} págs."
                             else:
                                 pages_str = "Páginas desconocidas"
                         except (ValueError, TypeError):
                              pages_str = "Páginas desconocidas"


                         # Start building the HTML for a single book item
                         html_content += f"""
                         <div style="flex: 0 0 auto; width: 150px; text-align: center; overflow-wrap: break-word; border: 1px solid #eee; padding: 10px; border-radius: 5px; background-color: #fff;">
                             <p style="font-size: 12px; font-weight: bold; margin-bottom: 5px; height: 3em; overflow: hidden; text-overflow: ellipsis;">{book_title}</p>
                             """

                         # Add the image tag or placeholder
                         if pd.notna(image_url) and image_url and isinstance(image_url, str):
                             html_content += f'<img src="{image_url}" alt="Portada de {book_title}" width="100" style="display: block; margin: 0 auto; border: 1px solid #ccc; height: 150px; object-fit: cover;">'
                         else:
                             html_content += f'<div style="width: 100px; height: 150px; border: 1px solid #ccc; display: flex; align-items: center; justify-content: center; font-size: 10px; background-color: #f0f0f0; margin: 0 auto;">No Cover Available</div>'

                         # Add other details
                         html_content += f"""
                             <p style="font-size: 10px; margin-top: 5px; color: #333;">Autor: {author}</p>
                             <p style="font-size: 10px; color: #333;">Género: {genre}</p>
                             <p style="font-size: 10px; color: #333;">{pages_str}</p>
                             <p style="font-size: 10px; font-weight: bold; color: green; margin-top: 5px;">Relevancia: {score:.4f}</p>
                         """

                         # Close the div for this book item
                         html_content += "</div>"


                 # Close the main container div
                 html_content += "</div>"

                 # Display the generated HTML
                 display(HTML(html_content))

        else:
            print(f"Error: El índice de usuario {user_index} está fuera de los límites de user_item_csr.shape[0] = {user_item_csr.shape[0]}.")

-------------------

Hasta aquí todo bien pero...
  
**Por qué neg_mean_squared_error no es la mejor métrica aquí?**  
  
El modelo NearestNeighbors no es un regresor; su propósito principal es encontrar los "vecinos" más cercanos en un espacio de datos, basándose en una métrica de distancia (como coseno, euclidiana, etc.). No predice un valor numérico (como una calificación específica) para un ítem. neg_mean_squared_error se usa para evaluar modelos que sí predicen valores numéricos continuos.

**Utilizaremos Precision@k y Recall@k**

1. **Precision@k** se centra en la "pureza" de la lista: ¿cuántas "basura" hay entre tus recomendaciones? Una alta precisión significa que la lista es muy relevante y el usuario no tendrá que "buscar" mucho para encontrar algo que le guste. *De los k (ej. 50) libros que tu sistema le recomendó a un usuario, ¿qué porcentaje de ellos fueron realmente relevantes para ese usuario?*
  
2. **Recall@k** se centra en la "exhaustividad" o "cobertura": ¿está tu sistema descubriendo todas las cosas que le gustarían al usuario? Un alto recall es importante para la "sorpresa" y para asegurar que el usuario no se pierda nada importante. *De todos los libros que un usuario realmente consideró relevantes (según tu umbral de rating, ej. 4.0 o más), ¿qué porcentaje de esos libros fueron capturados por tu sistema en el top k (ej. 50) de recomendaciones?*
  
Precision y Recall, al enfocarse en el "top-k", evalúan directamente la calidad de la lista final que el usuario ve.

----------

**Valor de la Esparsidad**
  
Una matriz muy dispersa dificulta que los algoritmos encuentren suficientes "vecinos" (usuarios o ítems similares) o aprendan patrones robustos, ya que hay muy poca información compartida directamente.

In [ ]:
user_item_matrix = df_ratings_modified.pivot_table(index='user_id', columns='book_id', values='rating').fillna(0)
user_item_csr_train = csr_matrix(user_item_matrix)

In [ ]:
total_elements_train = user_item_csr_train.shape[0] * user_item_csr_train.shape[1]
non_zero_elements_train = user_item_csr_train.nnz # Número de elementos no nulos (calificaciones)
sparsity_train = (1 - (non_zero_elements_train / total_elements_train)) * 100

print(f"\nDimensiones de la matriz de calificaciones: {user_item_csr_train.shape}")
print(f"Número total de celdas en la matriz: {total_elements_train}")
print(f"Número de calificaciones reales (celdas no nulas): {non_zero_elements_train}")
print(f"**Esparsidad de la matriz de calificaciones: {sparsity_train:.2f}%**")

Un alto porcentaje de esparsidad significa que la gran mayoría de las celdas de esa tabla están vacías (o contienen un 0, indicando que el usuario no ha calificado ese libro). Un 99.87% de esparsidad significa que solo el 0.11% de todas las posibles calificaciones en tu dataset han sido realmente dadas por los usuarios.
  
*De cada 10,000 posibles combinaciones de usuario y libro, ¡solo 11 tienen una calificación real!*

In [ ]:
# --- Bloque 1 (SURPRISE - SVD - V1.0): Preparación de Datos para Surprise ---

import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import SVD # Importamos el algoritmo SVD

print("--- Ejecutando Bloque 1 (SURPRISE - SVD - V1.0): Preparación de Datos ---")

# Asegúrate de que df_ratings_modified ya está cargado y disponible.
# Por ejemplo: df_ratings_modified = pd.read_csv('tu_archivo_de_ratings.csv')
# Si ya lo cargaste al principio de tu notebook, no necesitas esta línea de nuevo.

# Define el formato de tus ratings
# Reader define el rango de tus calificaciones (ej. de 1 a 5)
# Basado en ejemplos anteriores, asumo que tus ratings van de 1 a 5.
reader = Reader(rating_scale=(1, 5))

# Carga los datos en el formato de Surprise Dataset
# Asegúrate de que las columnas de df_ratings_modified sean 'user_id', 'book_id', 'rating'
# o ajusta los nombres de las columnas en load_from_df si son diferentes
data = Dataset.load_from_df(df_ratings_modified[['user_id', 'book_id', 'rating']], reader)

print(f"Datos cargados en el formato de Surprise. Total de calificaciones: {len(df_ratings_modified)}")
print("--- Bloque 1 (SURPRISE - SVD - V1.0) Completado ---")

In [ ]:
# --- Bloque 2 (SURPRISE - SVD - V1.0): Entrenamiento y Evaluación Básica (CORREGIDO) ---

print("\n--- Ejecutando Bloque 2 (SURPRISE - SVD - V1.0): Entrenamiento y Evaluación Básica ---")

# Inicializa el algoritmo SVD
algo_svd = SVD(n_factors=50, n_epochs=20, random_state=42)

# Realiza la validación cruzada
results = cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

print(f"\nResultados de la Validación Cruzada (SVD):")
print(f"  RMSE promedio: {results['test_rmse'].mean():.4f}")
print(f"  MAE promedio: {results['test_mae'].mean():.4f}")

# --- ¡CORRECCIÓN AQUÍ! Usamos np.mean() directamente ---
import numpy as np # Asegurarse de que numpy está importado

print(f"  Tiempo de entrenamiento promedio: {np.mean(results['fit_time']):.2f} segundos")
print(f"  Tiempo de prueba promedio: {np.mean(results['test_time']):.2f} segundos")

print("--- Bloque 2 (SURPRISE - SVD - V1.0) Completado ---")

**Resultados de Predicciones y Recomendaciones con SVD:**

En este bloque, el modelo SVD fue entrenado con todos los datos disponibles, lo que le permite generar predicciones para cualquier par usuario-libro. Por ejemplo, para el **usuario 47460 y el libro 22749994, la calificación estimada es de 3.86**, lo que demuestra la capacidad del modelo para predecir el agrado de un usuario por un libro específico.


In [ ]:
# --- Bloque 3 (SURPRISE - SVD - V1.0): Entrenamiento Final y Predicciones ---

print("\n--- Ejecutando Bloque 3 (SURPRISE - SVD - V1.0): Entrenamiento Final y Predicciones ---")

# Entrenar el modelo en todo el dataset de Surprise (útil si no vas a hacer más validación)
# build_full_trainset() crea un trainset a partir de todos los datos disponibles.
trainset = data.build_full_trainset()
algo_svd.fit(trainset)

print("Modelo SVD entrenado en el conjunto completo de datos.")

# --- Ejemplo de predicción para un usuario y un ítem ---
# Obtenemos IDs de usuario y libro de ejemplo directamente de tu DataFrame original
sample_user_id = df_ratings_modified['user_id'].sample(1, random_state=1).iloc[0]
sample_book_id = df_ratings_modified['book_id'].sample(1, random_state=2).iloc[0]

# Predice la calificación que el usuario daría a un libro
# Surprise espera IDs de usuario e ítem como strings (aunque sean números, los trata como identificadores únicos)
predicted_rating = algo_svd.predict(str(sample_user_id), str(sample_book_id))
print(f"\nPredicción para el usuario {sample_user_id} y el libro {sample_book_id}: {predicted_rating.est:.2f}")

# --- Función para Generar TOP-N recomendaciones para un usuario ---
# (Similar a lo que hacíamos antes para el debug, pero adaptado a Surprise)
def get_top_n_recommendations_surprise(algo, uid, df_ratings_original, n=10):
    # Obtener todos los libros que el usuario ya ha calificado
    # Asegúrate de que los IDs aquí coincidan con el tipo que espera Surprise (str)
    rated_book_ids = df_ratings_original.loc[df_ratings_original['user_id'] == uid, 'book_id'].tolist()

    # Obtener todos los libros únicos en el dataset (que el modelo conoce)
    all_book_ids = df_ratings_original['book_id'].unique()

    # Libros que el usuario NO ha calificado
    unrated_book_ids = [bid for bid in all_book_ids if bid not in rated_book_ids]

    predictions = []
    # Realizar predicciones para todos los libros no calificados
    # Convierte los IDs a str ya que Surprise los maneja así internamente.
    for book_id in unrated_book_ids:
        predictions.append(algo.predict(str(uid), str(book_id)))

    # Ordenar las predicciones por calificación estimada (descendente)
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Obtener los top N
    top_n_recs = predictions[:n]
    
    # Formatear para una mejor lectura y extraer solo los IDs de los libros
    formatted_recs = []
    top_n_ids = []
    for pred in top_n_recs:
        formatted_recs.append(f"  - Libro ID: {pred.iid}, Estimación: {pred.est:.2f}")
        top_n_ids.append(pred.iid) # Guardar el ID del libro
    
    return formatted_recs, top_n_ids


# Prueba para un usuario de ejemplo
user_id_for_recs = df_ratings_modified['user_id'].sample(1, random_state=3).iloc[0] # Otro usuario de ejemplo
top_n_formatted, top_n_ids = get_top_n_recommendations_surprise(algo_svd, user_id_for_recs, df_ratings_modified, n=10)

print(f"\nTop 10 recomendaciones para el usuario {user_id_for_recs} (usando SVD):")
if top_n_formatted:
    for rec in top_n_formatted:
        print(rec)
else:
    print("  (No se pudieron generar recomendaciones para este usuario o la lista está vacía)")


print("--- Bloque 3 (SURPRISE - SVD - V1.0) Completado ---")

Además, se generó una lista de **10 recomendaciones principales** para el **usuario 47460**. Estas son libros que el usuario aún no ha calificado, y el modelo estima que les gustarían. En este caso, todas las recomendaciones tienen una estimación de **3.86**, indicando que el modelo las considera igualmente atractivas para este usuario. Esto confirma que el SVD es capaz de generar listas de recomendaciones relevantes, superando los desafíos de la alta esparsidad de nuestros datos.

In [ ]:
# --- Bloque 4 (SURPRISE - SVD - V1.0): Evaluación de Precision@K y Recall@K ---

from collections import defaultdict # Necesario para la función
import numpy as np # Necesario para np.mean

print("\n--- Ejecutando Bloque 4 (SURPRISE - SVD - V1.0): Evaluación de Precision@K y Recall@K ---")

# --- Paso 1: División explícita de los datos (Train/Test Split para evaluación) ---
# Usamos un split de 80% entrenamiento / 20% prueba, como antes.
trainset_eval, testset_eval = train_test_split(data, test_size=0.2, random_state=42)

print(f"Datos divididos: {trainset_eval.n_ratings} ratings en entrenamiento, {len(testset_eval)} ratings en prueba.")

# --- Paso 2: Entrenar el modelo SVD en el conjunto de entrenamiento ---
# Utilizamos una nueva instancia del modelo por si queremos ajustar parámetros aquí.
algo_svd_eval = SVD(n_factors=50, n_epochs=20, random_state=42)
algo_svd_eval.fit(trainset_eval)

print("Modelo SVD entrenado en el conjunto de entrenamiento para evaluación.")

# --- Paso 3: Generar predicciones para el conjunto de prueba ---
# Las predicciones son una lista de objetos Prediction (uid, iid, r_ui, est, details)
predictions = algo_svd_eval.test(testset_eval)

print(f"Predicciones generadas para {len(predictions)} ítems en el conjunto de prueba.")

# --- Paso 4: Adaptar la función de evaluación (Precision@K y Recall@K) ---

def get_top_n_for_precision_recall(predictions, n=10, min_rating_threshold=2.5):
    """
    Retorna los top-N recomendaciones y los ítems relevantes para cada usuario
    a partir de una lista de predicciones.

    Args:
        predictions (list): Lista de objetos Prediction generados por model.test().
        n (int): El número de recomendaciones a considerar para Precision@K y Recall@K.
        min_rating_threshold (float): El umbral de calificación para considerar un ítem relevante.

    Returns:
        tuple: (top_n_recs, relevant_items)
            top_n_recs (defaultdict): {user_id: [item1, item2, ...]}
            relevant_items (defaultdict): {user_id: [relevant_item1, relevant_item2, ...]}
    """
    # Mapear predicciones a cada usuario
    user_preds = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_preds[uid].append((iid, est, true_r))

    top_n_recs = defaultdict(list)
    relevant_items = defaultdict(list)

    for uid, user_ratings in user_preds.items():
        # Obtener los top-N ítems recomendados (los que tienen mayor 'est' - estimación)
        user_ratings.sort(key=lambda x: x[1], reverse=True) # Ordenar por estimación
        top_n_recs[uid] = [iid for (iid, est, true_r) in user_ratings[:n]]

        # Obtener los ítems realmente relevantes (los que el usuario calificó alto en el testset)
        relevant_items[uid] = [iid for (iid, est, true_r) in user_ratings if true_r >= min_rating_threshold]

    return top_n_recs, relevant_items

def calculate_precision_recall(top_n_recs, relevant_items):
    precisions = dict()
    recalls = dict()

    for uid, rec_items in top_n_recs.items():
        rel_items = relevant_items.get(uid, []) # Items relevantes reales para este usuario
        
        n_relevant_and_recommended = len(set(rec_items) & set(rel_items)) # Intersección
        
        # Precision: Cuántas de las recomendaciones son relevantes
        if len(rec_items) > 0:
            precisions[uid] = n_relevant_and_recommended / len(rec_items)
        else:
            precisions[uid] = 0.0
            
        # Recall: Cuántos de los ítems relevantes fueron recomendados
        if len(rel_items) > 0:
            recalls[uid] = n_relevant_and_recommended / len(rel_items)
        else:
            recalls[uid] = 0.0 # Si no hay items relevantes reales, recall es 0

    return precisions, recalls

# --- Ejecución de la evaluación ---
# k_recs: El número de recomendaciones para calcular P@k y R@k
# Puedes ajustarlo para ver cómo cambia el rendimiento con más o menos recomendaciones.
k_recs = 10 # Evaluar Precision@10 y Recall@10 (o 20 si quieres ver un rango más amplio)
threshold_relevance = 2.5 # Umbral para considerar un rating como "relevante"

top_n_recommendations_svd, relevant_items_svd = get_top_n_for_precision_recall(
    predictions, n=k_recs, min_rating_threshold=threshold_relevance
)

precisions_svd, recalls_svd = calculate_precision_recall(
    top_n_recommendations_svd, relevant_items_svd
)

# Calcula promedios
avg_precision_svd = np.mean(list(precisions_svd.values())) if precisions_svd else 0.0
avg_recall_svd = np.mean(list(recalls_svd.values())) if recalls_svd else 0.0

print(f"\n--- Métricas de Recomendación para SVD (k={k_recs}, Umbral={threshold_relevance}) ---")
print(f"Precision@{k_recs} promedio: {avg_precision_svd:.4f}")
print(f"Recall@{k_recs} promedio: {avg_recall_svd:.4f}")

# Opcional: Debug para unos pocos usuarios si quieres ver el detalle
print("\n--- Detalles de depuración para los primeros 5 usuarios con items en el testset ---")
debug_count = 0
for uid in sorted(top_n_recommendations_svd.keys()):
    if debug_count >= 5:
        break
    
    recs = top_n_recommendations_svd[uid]
    rel_items = relevant_items_svd.get(uid, [])
    
    n_relevant_and_recommended = len(set(recs) & set(rel_items))

    precision_user = precisions_svd.get(uid, 0.0)
    recall_user = recalls_svd.get(uid, 0.0)

    print(f"\nUsuario: {uid}")
    print(f"  Top {k_recs} recomendaciones: {recs}")
    print(f"  Ítems verdaderamente relevantes (en TEST): {rel_items}")
    print(f"  Coincidencias relevantes y recomendadas: {n_relevant_and_recommended}")
    print(f"  Precision@{k_recs}: {precision_user:.4f}")
    print(f"  Recall@{k_recs}: {recall_user:.4f}")
    debug_count += 1


print("\n--- Bloque 4 (SURPRISE - SVD - V1.0) Completado ---")

Los valores de Precision@10 promedio: 0.8907 y Recall@10 promedio: 0.9388 son excepcionalmente altos, especialmente para un dataset con una esparsidad del 99.89%. Esto significa:
  
1. **Precision@10 (0.8907)**: Aproximadamente el 89% de los 10 libros recomendados por el modelo SVD son realmente libros que el usuario valoró positivamente en el conjunto de prueba.
  
2. **Recall@10 (0.9388)**: El modelo está logrando identificar y recomendar casi el 94% de todos los libros que un usuario valoró positivamente en el conjunto de prueba. Esto indica que el modelo es muy bueno encontrando la mayoría de los "libros relevantes ocultos".

In [ ]:
# --- Bloque 5 (SURPRISE - SVD - V1.0): Optimización de Hiperparámetros con GridSearchCV ---

from surprise.model_selection import GridSearchCV
from surprise import SVD # Asegúrate de que SVD esté importado aquí también si no lo está globalmente

print("\n--- Ejecutando Bloque 5 (SURPRISE - SVD - V1.0): Optimización de Hiperparámetros ---")

# Define el diccionario de parámetros a probar
# Puedes ajustar estos rangos. ¡Cuidado! Demasiadas combinaciones pueden tardar mucho.
# Si quieres probar un rango más pequeño al principio para ver cómo funciona, es buena idea.
param_grid = {
    'n_factors': [50, 100], # Número de factores latentes (dimensiones ocultas)
    'n_epochs': [20, 30],  # Número de iteraciones de entrenamiento
    'lr_all': [0.005, 0.01], # Tasa de aprendizaje
    'reg_all': [0.02, 0.05] # Término de regularización (para evitar overfitting)
}

# Inicializa GridSearchCV
# Establecemos measures=['rmse', 'mae'] para evaluar ambos, pero buscará el mejor 'rmse' por defecto
# cv=3 para hacer 3-fold cross-validation para cada combinación
# n_jobs=-1 para usar todos los núcleos de la CPU disponibles y acelerar el proceso
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

# Ejecuta la búsqueda en rejilla
# data es tu Dataset de Surprise (cargado en el Bloque 1)
gs.fit(data)

print("\n--- Búsqueda de Hiperparámetros Completada ---")

# --- Muestra los mejores resultados ---
print(f"Mejor RMSE: {gs.best_score['rmse']:.4f}")
print(f"Mejores parámetros para RMSE: {gs.best_params['rmse']}")

print(f"\nMejor MAE: {gs.best_score['mae']:.4f}")
print(f"Mejores parámetros para MAE: {gs.best_params['mae']}")

# Puedes guardar el mejor estimador para usarlo después en tus recomendaciones
best_svd_algo = gs.best_estimator['rmse']
print("\nEl mejor estimador SVD (basado en RMSE) ha sido guardado en la variable 'best_svd_algo'.")


print("\n--- Bloque 5 (SURPRISE - SVD - V1.0) Completado ---")

In [ ]:
# --- Bloque 6 (Interfaz): Preparación y Carga del Mejor Modelo SVD para Nuevo Usuario ---

import pandas as pd
import numpy as np # Necesario para np.random.choice
from surprise import Dataset, Reader, SVD
from collections import defaultdict

print("--- Ejecutando Bloque 6 (Interfaz): Preparación y Carga del Mejor Modelo para Nuevo Usuario ---")

# 1. Definir el nuevo user_id de forma consistente
# Asegúrate de que df_users está cargado desde el inicio del notebook
max_user_id = df_users['user_id'].max()
new_user_id = max_user_id + 1 # Un ID de usuario que sabemos que no existe en tu dataset original

print(f"ID del Nuevo Usuario a simular: {new_user_id}")

# 2. Definimos el Reader para nuestro rango de ratings
reader = Reader(rating_scale=(1, 5))

# --- MODIFICADO: Simular algunas calificaciones iniciales para el nuevo usuario con Book IDs existentes ---
# ESTO ES NECESARIO para que el modelo SVD tenga datos para aprender los factores latentes del nuevo usuario
# al re-entrenarse.

# Obtener una lista de Book IDs existentes en tu catálogo
existing_book_ids = df_combined_books['book_id'].unique()

# Asegurarse de que tenemos suficientes libros para simular
num_simulated_ratings = 3 # Número de calificaciones a simular
if len(existing_book_ids) < num_simulated_ratings:
    print(f"Advertencia: Solo hay {len(existing_book_ids)} libros únicos. Simularé con todos ellos.")
    simulated_book_ids = existing_book_ids
else:
    # Elegir aleatoriamente N book_ids únicos de los existentes
    simulated_book_ids = np.random.choice(existing_book_ids, size=num_simulated_ratings, replace=False)

simulated_ratings_data = []
for book_id in simulated_book_ids:
    # Asignar una calificación aleatoria entre 3 y 5 estrellas para dar una base positiva
    simulated_ratings_data.append((new_user_id, book_id, np.random.randint(3, 6))) # Calificación entre 3 y 5

print(f"Calificaciones simuladas para el nuevo usuario (para inicializar SVD): {simulated_ratings_data}")

# Convertir las calificaciones simuladas a un DataFrame
simulated_df = pd.DataFrame(simulated_ratings_data, columns=['user_id', 'book_id', 'rating'])

# 3. Combinar las calificaciones simuladas con el dataset original para el re-entrenamiento
# ¡Importante! Asegúrate de que 'df_ratings_modified' (tu DataFrame original de ratings) está disponible.
all_ratings_df = pd.concat([df_ratings_modified, simulated_df], ignore_index=True)

# 4. Cargar el DataFrame combinado en el formato de Surprise
full_data_with_new_user = Dataset.load_from_df(all_ratings_df[['user_id', 'book_id', 'rating']], reader)

# 5. Construir el trainset completo que incluye al nuevo usuario
trainset_with_new_user = full_data_with_new_user.build_full_trainset()

# 6. Re-entrenar el mejor modelo SVD con los datos actualizados, incluyendo el nuevo usuario
# Asegúrate de que 'best_svd_algo' fue generado por el Bloque 5.
print(f"Re-entrenando el modelo SVD optimizado con el nuevo usuario {new_user_id} y sus calificaciones simuladas...")
best_svd_algo.fit(trainset_with_new_user)
print("Modelo SVD re-entrenado con éxito.")


print("\n--- Preparación para la Interfaz Completada. El modelo SVD está listo para usar el nuevo usuario. ---")

### **5.4.1 PONIENDO EL RECOMENDADOR A PRUEBA CON UN CASO "REAL"**

In [ ]:
# # --- Bloque 7 (Interfaz): Interfaz Interactiva de Recomendación con SVD ---

# import ipywidgets as widgets
# from IPython.display import display, HTML
# import pandas as pd
# import numpy as np
# from scipy.sparse import csr_matrix
# from surprise import Dataset, Reader, SVD
# from collections import defaultdict

# print("\n--- Ejecutando Bloque 7 (Interfaz): Interfaz Interactiva de Recomendación con SVD ---")

# # GLOBAL_VAR_REQUIRED: Asegúrate de que estas variables estén disponibles desde bloques anteriores
# # df_combined_books
# # df_users
# # best_svd_algo (el modelo SVD optimizado del Bloque 5, re-entrenado con el nuevo usuario en Bloque 6)
# # simulated_df (el DataFrame con las calificaciones simuladas del nuevo usuario del Bloque 6)
# # df_ratings_modified (el DataFrame de calificaciones original, pre-procesado)
# # user_item_matrix (la matriz usuario-ítem para la parte de KNN)
# # knn_best (el modelo NearestNeighbors entrenado para la parte de KNN)


# # --- MODIFICADO: Asegurarse de que la columna 'genre' es string y manejar NaNs/vacíos ---
# if 'genre' in df_combined_books.columns:
#     df_combined_books['genre'] = df_combined_books['genre'].astype(str).replace('nan', '').replace('', 'Género Desconocido')
# else:
#     print("Advertencia: No se encontró la columna 'genre' en df_combined_books.")
#     df_combined_books['genre'] = 'Género Desconocido'

# unique_genres = df_combined_books['genre'].unique().tolist()
# unique_genres_selector = sorted([g for g in unique_genres if g and pd.notna(g) and g != 'Género Desconocido'])


# # --- Función para determinar el género preferido (sin cambios) ---
# def assign_reading_preference(ratings_df_temp, books_df, unique_genres_list):
#     """
#     Determina el género preferido basado en las calificaciones proporcionadas (en ratings_df_temp).
#     """
#     user_books_rated_now = ratings_df_temp['book_id']
#     user_genres = books_df[books_df['book_id'].isin(user_books_rated_now)]['genre']

#     if not user_genres.empty:
#         # Divide los géneros múltiples y aplana la lista para contar correctamente
#         all_rated_genres = user_genres.apply(lambda x: [g.strip() for g in str(x).split(',') if g.strip()]).explode()
        
#         if not all_rated_genres.empty and not all_rated_genres.mode().empty:
#             most_frequent_genre = all_rated_genres.mode().iloc[0]
#             if pd.notna(most_frequent_genre) and most_frequent_genre.strip() != "" and most_frequent_genre != 'Género Desconocido':
#                 return most_frequent_genre.strip()
#             # Si el más frecuente es "Desconocido" o vacío, intenta con el siguiente si hay
#             elif len(all_rated_genres.mode()) > 1:
#                 second_most_frequent = all_rated_genres.mode().iloc[1]
#                 if pd.notna(second_most_frequent) and second_most_frequent.strip() != "" and second_most_frequent != 'Género Desconocido':
#                     return second_most_frequent.strip()

#         # Si aún no se encontró un género válido, elige uno aleatorio de los válidos
#         valid_genres_in_list = [g for g in unique_genres_list if g and pd.notna(g) and g != 'Género Desconocido']
#         if valid_genres_in_list:
#             return np.random.choice(valid_genres_in_list)
#         else:
#             return "Preferencias desconocidas"
#     else:
#         # Si no hay calificaciones, elige un género aleatorio
#         valid_genres_in_list = [g for g in unique_genres_list if g and pd.notna(g) and g != 'Género Desconocido']
#         if valid_genres_in_list:
#             return np.random.choice(valid_genres_in_list)
#         else:
#             return "Preferencias desconocidas"


# # --- Configuración de Estilo Global (sin cambios) ---
# background_color = "#f7eecd"
# font_family = "'Roboto', 'Open Sans', 'Segoe UI', 'Arial', sans-serif"
# base_container_style = f"background-color: {background_color}; font-family: {font_family}; padding: 15px; border-radius: 8px;"
# text_style = f"color: #333; font-family: {font_family};"
# light_text_style = f"color: #555; font-family: {font_family};"


# print("--- Configuración de Nuevo Usuario Interactivo ---")

# # --- 1. Crear un Nuevo User_ID y Perfil (Inicial - sin preferencia aún) ---
# max_user_id = df_users['user_id'].max()
# new_user_id = max_user_id + 1 # Usamos un ID muy alto para asegurar que no colisiona con usuarios existentes

# new_user_data_initial = {
#     'user_id': new_user_id,
#     'avatar_url': f"https://api.dicebear.com/7.x/identicon/svg?seed={new_user_id}",
#     'username': f'NuevoUsuario_{new_user_id}'
# }

# print(f"Configurando Nuevo Usuario '{new_user_data_initial['username']}' (ID: {new_user_id}).")
# print("Por favor, selecciona sus características y califica algunos libros.")


# # --- 2. Preparar Widgets Interactivos (Añadidos Desplegables de Perfil) ---

# # Obtener opciones únicas de df_users, manejando NaNs
# def get_unique_options(df, column):
#     if column in df.columns:
#         options = df[column].dropna().astype(str).unique().tolist()
#         options.sort()
#         options = [opt for opt in options if str(opt).strip() != '']
#         return [f"--- Selecciona {column.capitalize()} ---"] + options
#     else:
#         print(f"Advertencia: Columna '{column}' no encontrada en df_users.")
#         return [f"--- No hay datos de {column.capitalize()} ---"]

# age_options = get_unique_options(df_users, 'age')
# age_dropdown = widgets.Dropdown(options=age_options, description='Edad:')

# gender_options = get_unique_options(df_users, 'gender')
# gender_dropdown = widgets.Dropdown(options=gender_options, description='Género:')

# education_options = get_unique_options(df_users, 'education')
# education_dropdown = widgets.Dropdown(options=education_options, description='Educación:')

# country_options = get_unique_options(df_users, 'country')
# country_dropdown = widgets.Dropdown(options=country_options, description='País:')


# # Desplegable de Géneros de Libros
# genre_options = unique_genres_selector.copy()
# genre_options.insert(0, "--- Selecciona un Género ---")

# genre_dropdown = widgets.Dropdown(
#     options=genre_options,
#     description='Género:',
#     disabled=False,
# )

# # Desplegable de Libros
# book_dropdown = widgets.Dropdown(
#     options=[("--- Selecciona un Género Primero ---", None)],
#     description='Libro:',
#     disabled=True,
# )

# # Desplegable de Calificación
# rating_options = [(str(i), i) for i in range(1, 6)]
# rating_dropdown = widgets.Dropdown(
#     options=rating_options,
#     description='Calificación:',
#     disabled=False,
# )

# # Botón para Añadir Calificación
# add_rating_button = widgets.Button(description="Añadir Calificación")
# add_rating_button.disabled = True

# # Área de salida para mostrar las calificaciones añadidas
# output_ratings = widgets.Output()

# # Lista para almacenar las calificaciones del nuevo usuario
# new_user_ratings_collected = []


# # --- 3. Lógica para Actualizar el Desplegable de Libros al Cambiar el Género (sin cambios) ---
# def on_genre_change(change):
#     selected_genre = change['new']
#     book_dropdown.disabled = True
#     add_rating_button.disabled = True
#     book_dropdown.options = [("--- Cargando Libros ---", None)]
#     book_dropdown.value = None

#     if selected_genre == "--- Selecciona un Género ---":
#         book_dropdown.options = [("--- Selecciona un Género Primero ---", None)]
#     else:
#         # Asegúrate de que df_combined_books['genre'] es string antes de usar .str.contains
#         filtered_books = df_combined_books[
#             df_combined_books['genre'].astype(str).str.contains(selected_genre, na=False, case=False)
#         ]

#         if not filtered_books.empty:
#             book_options_filtered = [(f"{row['title']} por {row['authors']}", row['book_id'])
#                                      for index, row in filtered_books.iterrows()]
#             book_options_filtered.insert(0, ("--- Selecciona un Libro ---", None))

#             book_dropdown.options = book_options_filtered
#             book_dropdown.disabled = False
#             if len(book_options_filtered) > 1:
#                 add_rating_button.disabled = False # Habilitar añadir calificación si hay libros para seleccionar

#         else:
#             book_dropdown.options = [("--- No se encontraron libros en este género ---", None)]


# genre_dropdown.observe(on_genre_change, names='value')


# # --- 4. Lógica para el Botón "Añadir Calificación" (sin cambios) ---
# def on_add_rating_button_clicked(b):
#     selected_book_id = book_dropdown.value
#     selected_rating = rating_dropdown.value
#     selected_book_title = book_dropdown.label

#     if selected_book_id is not None and selected_rating is not None:
#         already_rated = any(d['book_id'] == selected_book_id for d in new_user_ratings_collected)

#         if not already_rated:
#             new_user_ratings_collected.append({
#                 'user_id': new_user_id,
#                 'book_id': selected_book_id,
#                 'rating': selected_rating
#             })
#             with output_ratings:
#                 print(f"Añadida calificación: '{selected_book_title}' con {selected_rating} estrellas.")
#         else:
#             with output_ratings:
#                 print(f"Advertencia: Ya calificaste '{selected_book_title}'.")

#     else:
#         with output_ratings:
#             print("Advertencia: Selecciona un libro y una calificación válidos.")

# add_rating_button.on_click(on_add_rating_button_clicked)


# # --- 5. Botón para Generar Recomendaciones y Lógica (ADAPTADA PARA EL ERROR 'get_params') ---
# recommend_button = widgets.Button(description="Generar Recomendaciones")
# output_recommendations = widgets.Output()
# output_similar_user_books = widgets.Output() # Área de salida dedicada para mostrar libros de usuarios similares clicados

# # --- Función para mostrar libros de un usuario similar clicado (sin cambios) ---
# def display_similar_user_rated_books(similar_user_id, target_user_rated_books_set, recommended_book_ids_set,
#                                      df_combined_books, user_item_matrix_param, output_widget):
#     """
#     Muestra los libros calificados por un usuario similar, excluyendo los que el
#     usuario objetivo ya calificó o los que ya fueron recomendados.
#     """
#     output_widget.clear_output()

#     with output_widget:
#         print(f"\nLibros calificados por Usuario Similar {similar_user_id} (excluyendo los tuyos y los ya recomendados):")

#         if similar_user_id not in user_item_matrix_param.index:
#             print(f"Info del usuario similar {similar_user_id} no encontrada en la matriz. Posiblemente no calificó nada en el dataset original.")
#             return

#         sim_user_ratings = user_item_matrix_param.loc[similar_user_id]
#         sim_user_rated_book_ids = set(sim_user_ratings[sim_user_ratings > 0].index)

#         books_to_show = sim_user_rated_book_ids - target_user_rated_books_set - recommended_book_ids_set

#         if not books_to_show:
#             print("Este usuario no calificó ningún libro que no tengas o que ya se te haya recomendado.")
#             return

#         books_details_to_show = df_combined_books[df_combined_books['book_id'].isin(books_to_show)]

#         if books_details_to_show.empty:
#             print("No se encontraron detalles para los libros calificados por este usuario.")
#             return

#         html_content = f"<div style='display: flex; flex-wrap: wrap; gap: 15px; {base_container_style}'>"

#         for index, details in books_details_to_show.iterrows():
#             book_id_int = int(details['book_id'])
#             book_title = details.get('title', "Título desconocido")
#             image_url = details.get('image_url')
#             author = details.get('authors', "Autor desconocido")
#             genre = details.get('genre', "Género desconocido")
#             pages = details.get('pages', "Páginas desconocidas")

#             pages_str = "Páginas desconocidas"
#             try:
#                 if pd.notna(pages):
#                     pages_str = f"{int(pages)} págs."
#             except (ValueError, TypeError):
#                 pass

#             html_content += f"""
#             <div style="flex: 0 0 auto; width: 150px; text-align: center; overflow-wrap: break-word; border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color: #fff; font-family: {font_family};">
#                 <p style="font-size: 12px; font-weight: bold; margin-bottom: 5px; height: 3em; overflow: hidden; text-overflow: ellipsis; {text_style}">{book_title}</p>
#                 <img src="{image_url}" alt="Portada de {book_title}" width="100" style="display: block; margin: 0 auto; border: 1px solid #ccc; height: 150px; object-fit: cover;">
#                 <p style="font-size: 10px; margin-top: 5px; {light_text_style}">Autor: {author}</p>
#                 <p style="font-size: 10px; {light_text_style}>Género: {genre}</p>
#                 <p style="font-size: 10px; {light_text_style}>{pages_str}</p>
#                 </div>
#             """

#         html_content += "</div>"
#         display(HTML(html_content))


# def on_recommend_button_clicked(b):
#     output_recommendations.clear_output()
#     output_similar_user_books.clear_output()

#     # --- NEW: Capturar valores de los nuevos desplegables de perfil ---
#     selected_age = age_dropdown.value if age_dropdown.value != '--- Selecciona Edad ---' else 'Desconocida'
#     selected_gender = gender_dropdown.value if gender_dropdown.value != '--- Selecciona Género ---' else 'Desconocido'
#     selected_education = education_dropdown.value if education_dropdown.value != '--- Selecciona Educación ---' else 'Desconocida'
#     selected_country = country_dropdown.value if country_dropdown.value != '--- Selecciona País ---' else 'Desconocido'

#     # Actualizar new_user_data_initial con los valores capturados
#     new_user_data = new_user_data_initial.copy()
#     new_user_data['age'] = selected_age
#     new_user_data['gender'] = selected_gender
#     new_user_data['education'] = selected_education
#     new_user_data['country'] = selected_country


#     if len(new_user_ratings_collected) < 3:
#         with output_recommendations:
#             print("Por favor, califica al menos 3 libros antes de generar recomendaciones.")
#         return

#     # Desactivar widgets durante la generación
#     add_rating_button.disabled = True
#     recommend_button.disabled = True
#     genre_dropdown.disabled = True
#     book_dropdown.disabled = True
#     rating_dropdown.disabled = True

#     with output_recommendations:
#         print("Generando recomendaciones...")
#         print("Re-entrenando SVD temporalmente con calificaciones actualizadas (esto puede tomar un momento)...")


#     try:
#         # Extraer listas de book_id y ratings de las calificaciones recogidas
#         rated_book_ids = [d['book_id'] for d in new_user_ratings_collected]
#         ratings = [d['rating'] for d in new_user_ratings_collected]

#         # --- MODIFICACIÓN CLAVE AQUÍ: Re-entrenar SVD con las calificaciones interactivas ---
#         # 1. Obtener los parámetros del modelo SVD optimizado del Bloque 5
#         svd_tuned_params = {}
#         try:
#             svd_tuned_params['n_factors'] = getattr(best_svd_algo, 'n_factors', 100)
#             svd_tuned_params['n_epochs'] = getattr(best_svd_algo, 'n_epochs', 20)
#             svd_tuned_params['lr_all'] = getattr(best_svd_algo, 'lr_all', 0.005)
#             svd_tuned_params['reg_all'] = getattr(best_svd_algo, 'reg_all', 0.02)
#             print(f"Parámetros SVD obtenidos de best_svd_algo: {svd_tuned_params}")

#         except AttributeError:
#             print("Advertencia: No se pudieron extraer los parámetros SVD de 'best_svd_algo' directamente.")
#             print("Usando valores de ejemplo (n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02).")
#             print("Por favor, reemplaza estos con los parámetros óptimos de tu SVD (ej. de tu GridSearch).")
#             svd_tuned_params = {
#                 'n_factors': 100,
#                 'n_epochs': 20,
#                 'lr_all': 0.005,
#                 'reg_all': 0.02
#             }
#         except Exception as e:
#             print(f"Ocurrió un error inesperado al intentar obtener parámetros SVD: {e}")
#             print("Usando valores de ejemplo.")
#             svd_tuned_params = {
#                 'n_factors': 100,
#                 'n_epochs': 20,
#                 'lr_all': 0.005,
#                 'reg_all': 0.02
#             }


#         # 2. Crear una NUEVA instancia de SVD con esos parámetros
#         dynamic_svd_algo = SVD(**svd_tuned_params)

#         # 3. Combinar TODAS las calificaciones relevantes:
#         all_current_ratings_df = pd.concat([df_ratings_modified, simulated_df, pd.DataFrame(new_user_ratings_collected)], ignore_index=True)

#         # 4. Crear un trainset a partir de estos datos combinados
#         reader = Reader(rating_scale=(1, 5))
#         current_full_data = Dataset.load_from_df(all_current_ratings_df[['user_id', 'book_id', 'rating']], reader)
#         current_trainset = current_full_data.build_full_trainset()

#         # 5. Re-entrenar el modelo SVD TEMPORAL con el trainset completo y actualizado
#         dynamic_svd_algo.fit(current_trainset)
#         print("Modelo SVD temporal re-entrenado con éxito.")


#         # --- CALCULAR READING PREFERENCE BASADA EN LAS CALIFICACIONES ---
#         temp_new_user_ratings_df = pd.DataFrame(new_user_ratings_collected)
#         calculated_reading_preference = assign_reading_preference(
#             temp_new_user_ratings_df, df_combined_books, unique_genres
#         )
#         new_user_data['reading_preference'] = calculated_reading_preference


#         # --- Encontrar Vecinos Similares (usando la lógica original de KNN) ---
#         all_book_ids_in_matrix = user_item_matrix.columns
#         num_books = len(all_book_ids_in_matrix)
#         book_id_to_col_index = {book_id: i for i, book_id in enumerate(all_book_ids_in_matrix)}

#         new_user_vector_dense = np.zeros(num_books)
#         for rated_book_id, rating in zip(rated_book_ids, ratings):
#             if rated_book_id in book_id_to_col_index:
#                 col_index = book_id_to_col_index[rated_book_id]
#                 new_user_vector_dense[col_index] = rating

#         new_user_sparse_vector = csr_matrix(new_user_vector_dense.reshape(1, -1))

#         n_neighbors_find = 15
#         distances, indices = knn_best.kneighbors(new_user_sparse_vector, n_neighbors=n_neighbors_find)

#         similarity_data = {'user_id': [], 'distance': []}
#         for i in range(indices.shape[1]):
#             user_idx = indices[0][i]
#             dist = distances[0][i]
#             original_user_id = user_item_matrix.index[user_idx]

#             similarity_data['user_id'].append(original_user_id)
#             similarity_data['distance'].append(dist)

#         similarity_df_for_recommendation = pd.DataFrame(similarity_data)

#         if knn_best.metric == 'cosine':
#             similarity_df_for_recommendation['similarity'] = 1 - similarity_df_for_recommendation['distance']
#         else:
#             similarity_df_for_recommendation['similarity'] = 1 / (1 + similarity_df_for_recommendation['distance'])

#         top_n_neighbors_for_recommendation = 10
#         similarity_df_for_recommendation = similarity_df_for_recommendation.sort_values(by='similarity', ascending=False).head(top_n_neighbors_for_recommendation)


#         if similarity_df_for_recommendation.empty:
#             with output_recommendations:
#                 print("No se encontraron vecinos adecuados para generar recomendaciones.")


#         # --- Lógica de SVD para Generar Recomendaciones de Libros (CON PRIORIZACIÓN DE GÉNERO) ---
#         all_book_ids = set(df_combined_books['book_id'].unique())
#         books_to_predict_for = list(all_book_ids - set(rated_book_ids))

#         predictions = []
#         for book_id in books_to_predict_for:
#             pred = dynamic_svd_algo.predict(new_user_id, book_id)
#             predictions.append((book_id, pred.est))

#         predictions.sort(key=lambda x: x[1], reverse=True)

#         top_books_svd_raw = predictions # Mantener todas las predicciones ordenadas

#         final_recommended_books_details = []
#         recommended_book_ids_set = set()

#         num_svd_top_to_take = 5 # Tomar los 5 mejores de SVD puro inicialmente
        
#         # 1. Tomar las N mejores recomendaciones puras de SVD
#         for book_id, estimated_score in top_books_svd_raw[:num_svd_top_to_take]:
#             book_details = df_combined_books[df_combined_books['book_id'] == int(book_id)]
#             if not book_details.empty:
#                 details = book_details.iloc[0]
#                 final_recommended_books_details.append((book_id, estimated_score, details))
#                 recommended_book_ids_set.add(book_id)

#         # 2. Añadir recomendaciones potenciadas por género (si hay una preferencia clara)
#         if new_user_data.get('reading_preference') and new_user_data['reading_preference'] != 'Desconocida':
#             pref_genre = new_user_data['reading_preference']
#             genre_boosted_candidates = []

#             for book_id, estimated_score in top_books_svd_raw: # Iterar a través de todas las predicciones SVD
#                 if book_id not in recommended_book_ids_set: # Solo considerar libros no añadidos ya
#                     book_details = df_combined_books[df_combined_books['book_id'] == int(book_id)]
#                     if not book_details.empty:
#                         details = book_details.iloc[0]
#                         # Verificar si el género preferido está en los géneros del libro (manejando múltiples géneros)
#                         book_genres = str(details.get('genre', '')).split(', ')
#                         if pref_genre in book_genres:
#                             genre_boosted_candidates.append((book_id, estimated_score, details))
            
#             # Ordenar los candidatos de género por puntuación
#             genre_boosted_candidates.sort(key=lambda x: x[1], reverse=True)

#             # Añadir hasta el total deseado (10 libros) de los candidatos potenciados por género
#             for book_id, estimated_score, details in genre_boosted_candidates:
#                 if len(final_recommended_books_details) < 10:
#                     final_recommended_books_details.append((book_id, estimated_score, details))
#                     recommended_book_ids_set.add(book_id)
#                 else:
#                     break
        
#         # 3. Rellenar los slots restantes con las siguientes mejores predicciones SVD (si quedan slots)
#         if len(final_recommended_books_details) < 10:
#             for book_id, estimated_score in top_books_svd_raw:
#                 if book_id not in recommended_book_ids_set:
#                     book_details = df_combined_books[df_combined_books['book_id'] == int(book_id)]
#                     if not book_details.empty:
#                         details = book_details.iloc[0]
#                         final_recommended_books_details.append((book_id, estimated_score, details))
#                         recommended_book_ids_set.add(book_id)
#                         if len(final_recommended_books_details) >= 10:
#                             break

#         # Ordenar la lista final por puntuación estimada para asegurar que los 10 primeros son los mejores
#         final_recommended_books_details.sort(key=lambda x: x[1], reverse=True)
#         final_recommended_books_details = final_recommended_books_details[:10] # Asegurar que solo hay 10


#         # --- Mostrar Resultados ---
#         with output_recommendations:
#             # Mostrar perfil del nuevo usuario
#             user_html = f"""
#             <div style="{base_container_style} margin-bottom: 20px;">
#                 <div style="display: flex; align-items: center; border-bottom: 1px solid #eee; padding-bottom: 15px;">
#                     <div style="margin-right: 20px;">
#                         <img src="{new_user_data['avatar_url']}" alt="Avatar de {new_user_data['username']}" width="100" style="border-radius: 50%; border: 2px solid #ccc; object-fit: cover;">
#                     </div>
#                     <div>
#                         <h3 style="margin: 0 0 5px 0; {text_style}">Perfil del Usuario Seleccionado: <span style="color: #007bff; font-weight: bold;">{new_user_data['username']}</span> (ID: {new_user_id})</h3>
#                         <p style="margin: 0; font-size: 14px; {light_text_style}">Edad: {new_user_data.get('age', 'Desconocida')}</p>
#                         <p style="margin: 0; font-size: 14px; {light_text_style}">Género: {new_user_data.get('gender', 'Desconocido')}</p>
#                         <p style="margin: 0; font-size: 14px; {light_text_style}">Educación: {new_user_data.get('education', 'Desconocida')}</p>
#                         <p style="margin: 0; font-size: 14px; {light_text_style}">País: {new_user_data.get('country', 'Desconocido')}</p>
#                         <p style="margin: 0; font-size: 14px; {light_text_style}">Preferencia de lectura: {new_user_data.get('reading_preference', 'Desconocida')}</p>
#                     </div>
#                 </div>
#             </div>
#             """
#             display(HTML(user_html))

#             num_similar_users_to_show_in_display = 4

#             # Asegúrate de que similarity_df_for_recommendation esté definida aquí
#             top_similar_users_for_display = similarity_df_for_recommendation.head(num_similar_users_to_show_in_display)
#             top_similar_user_ids_for_display = top_similar_users_for_display['user_id'].tolist()

#             similar_user_widgets_list = []

#             top_similar_users_info_display = df_users[df_users['user_id'].isin(top_similar_user_ids_for_display)].set_index('user_id')


#             if not top_similar_users_info_display.empty:
#                 similar_users_title_html = f"<div style='{base_container_style} margin-bottom: 10px;'>"
#                 similar_users_title_html += f"<h3 style='margin-top: 0; margin-bottom: 5px; {text_style}'>Usuarios Similares (Haz clic para ver sus libros):</h3>"
#                 similar_users_title_html += "</div>"
#                 display(HTML(similar_users_title_html))

#                 similar_users_hbox_container = widgets.HBox([])
#                 similar_users_hbox_container.layout.justify_content = 'flex-start'
#                 similar_users_hbox_container.layout.margin = '0 0 10px 0'


#                 for sim_user_id in top_similar_users_for_display['user_id']:
#                     sim_user_info = top_similar_users_info_display.loc[sim_user_id] if sim_user_id in top_similar_users_info_display.index else None

#                     if sim_user_info is not None:
#                         sim_user_avatar_url = sim_user_info.get('avatar_url', f"https://api.dicebear.com/7.x/identicon/svg?seed={sim_user_id}")
#                         sim_user_username = sim_user_info.get('username', f'Usuario {sim_user_id}')

#                         avatar_html_value = f"""
#                         <div style="text-align: center; width: 80px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap; font-family: {font_family};">
#                             <img src="{sim_user_avatar_url}" alt="Portada de {sim_user_username}" width="50" height="50" style="border-radius: 50%; border: 1px solid #ccc; object-fit: cover; display: block; margin: 0 auto;"><br>
#                             <span style="font-size: 10px; {light_text_style}" title="{sim_user_username}">{sim_user_username}</span>
#                         </div>
#                         """
#                         avatar_widget_html = widgets.HTML(value=avatar_html_value)

#                         action_button = widgets.Button(
#                             description='',
#                             layout=widgets.Layout(width='80px', height='80px', margin='-80px 0 0 0'),
#                             style=widgets.ButtonStyle(
#                                 button_color='transparent',
#                                 border='1px solid transparent',
#                                 cursor='pointer'
#                             )
#                         )
#                         action_button._similar_user_id = sim_user_id

#                         action_button.on_click(lambda b: display_similar_user_rated_books(
#                             b._similar_user_id,
#                             set(rated_book_ids),
#                             recommended_book_ids_set,
#                             df_combined_books,
#                             user_item_matrix,
#                             output_similar_user_books
#                         ))

#                         user_widget_vbox = widgets.VBox([avatar_widget_html, action_button], layout=widgets.Layout(align_items='center', margin='0', padding='0', width='90px'))

#                         similar_user_widgets_list.append(user_widget_vbox)

#                 if similar_user_widgets_list:
#                     similar_users_hbox_container.children = similar_user_widgets_list
#                     display(similar_users_hbox_container)
#                 else:
#                     print("No se pudieron crear widgets para usuarios similares.")
#             else:
#                 print("\nNo se encontraron usuarios similares para mostrar.")

#             print(f"\n--- Top 10 Libros Recomendados para {new_user_data['username']} (ID: {new_user_id}): ---")

#             if not final_recommended_books_details:
#                 print("No se encontraron recomendaciones de libros.")
#             else:
#                 html_content = f"<div style='display: flex; flex-wrap: wrap; gap: 15px; {base_container_style}'>"

#                 for book_id, estimated_score, details in final_recommended_books_details:
#                     book_title = details.get('title', "Título desconocido")
#                     image_url = details.get('image_url')
#                     author = details.get('authors', "Autor desconocido")
#                     genre = details.get('genre', "Género desconocido")
#                     pages = details.get('pages', "Páginas desconocidas")

#                     pages_str = "Páginas desconocidas"
#                     try:
#                         if pd.notna(pages):
#                             pages_str = f"{int(pages)} págs."
#                     except (ValueError, TypeError):
#                         pass

#                     html_content += f"""
#                     <div style="flex: 0 0 auto; width: 150px; text-align: center; overflow-wrap: break-word; border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color: #fff; font-family: {font_family};">
#                         <p style="font-size: 12px; font-weight: bold; margin-bottom: 5px; height: 3em; overflow: hidden; text-overflow: ellipsis; {text_style}">{book_title}</p>
#                         <img src="{image_url}" alt="Portada de {book_title}" width="100" style="display: block; margin: 0 auto; border: 1px solid #ccc; height: 150px; object-fit: cover;">
#                         <p style="font-size: 10px; margin-top: 5px; {light_text_style}>Autor: {author}</p>
#                         <p style="font-size: 10px; {light_text_style}>Género: {genre}</p>
#                         <p style="font-size: 10px; {light_text_style}>{pages_str}</p>
#                         <p style="font-size: 10px; font-weight: bold; color: green; margin-top: 5px; font-family: {font_family};">Estimación: {estimated_score:.2f}</p>
#                     </div>
#                     """

#                 html_content += "</div>"
#                 display(HTML(html_content))


#     except Exception as e:
#         with output_recommendations:
#             print(f"Ocurrió un error al generar recomendaciones: {e}")

#     finally:
#         # Reactivar widgets al finalizar
#         add_rating_button.disabled = False
#         recommend_button.disabled = False
#         genre_dropdown.disabled = False
#         if genre_dropdown.value != "--- Selecciona un Género ---":
#             book_dropdown.disabled = False
#             if book_dropdown.value is not None:
#                 add_rating_button.disabled = False
#         rating_dropdown.disabled = False


# # Vincular la lógica al evento click del botón de recomendación
# recommend_button.on_click(on_recommend_button_clicked)


# # --- 6. Mostrar los Widgets Interactivos ---
# print("\nSelecciona las características del usuario, un género y califica libros:")
# interactive_widgets = widgets.VBox([
#     age_dropdown,
#     gender_dropdown,
#     education_dropdown,
#     country_dropdown,

#     genre_dropdown,
#     book_dropdown,
#     rating_dropdown,
#     add_rating_button,
#     output_ratings,
#     recommend_button,
#     output_recommendations,
#     output_similar_user_books
# ])

# display(interactive_widgets)

# print("\n--- Bloque 7 (Interfaz) Completado ---")

In [ ]:
# --- Bloque 7 (Interfaz): Interfaz Interactiva de Recomendación con SVD ---

import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from surprise import Dataset, Reader, SVD
from collections import defaultdict

print("\n--- Ejecutando Bloque 7 (Interfaz): Interfaz Interactiva de Recomendación con SVD ---")

# GLOBAL_VAR_REQUIRED: Asegúrate de que estas variables estén disponibles desde bloques anteriores
# df_combined_books
# df_users
# best_svd_algo (el modelo SVD optimizado del Bloque 5, re-entrenado con el nuevo usuario en Bloque 6)
# simulated_df (el DataFrame con las calificaciones simuladas del nuevo usuario del Bloque 6)
# df_ratings_modified (el DataFrame de calificaciones original, pre-procesado)
# user_item_matrix (la matriz usuario-ítem para la parte de KNN)
# knn_best (el modelo NearestNeighbors entrenado para la parte de KNN)


# --- MODIFICADO: Asegurarse de que la columna 'genre' es string y manejar NaNs/vacíos ---
if 'genre' in df_combined_books.columns:
    df_combined_books['genre'] = df_combined_books['genre'].astype(str).replace('nan', '').replace('', 'Género Desconocido')
else:
    print("Advertencia: No se encontró la columna 'genre' en df_combined_books.")
    df_combined_books['genre'] = 'Género Desconocido'

unique_genres = df_combined_books['genre'].unique().tolist()
unique_genres_selector = sorted([g for g in unique_genres if g and pd.notna(g) and g != 'Género Desconocido'])


# --- Función para determinar el género preferido (sin cambios) ---
def assign_reading_preference(ratings_df_temp, books_df, unique_genres_list):
    """
    Determina el género preferido basado en las calificaciones proporcionadas (en ratings_df_temp).
    """
    user_books_rated_now = ratings_df_temp['book_id']
    user_genres = books_df[books_df['book_id'].isin(user_books_rated_now)]['genre']

    if not user_genres.empty:
        # Divide los géneros múltiples y aplana la lista para contar correctamente
        all_rated_genres = user_genres.apply(lambda x: [g.strip() for g in str(x).split(',') if g.strip()]).explode()
        
        if not all_rated_genres.empty and not all_rated_genres.mode().empty:
            most_frequent_genre = all_rated_genres.mode().iloc[0]
            if pd.notna(most_frequent_genre) and most_frequent_genre.strip() != "" and most_frequent_genre != 'Género Desconocido':
                return most_frequent_genre.strip()
            # Si el más frecuente es "Desconocido" o vacío, intenta con el siguiente si hay
            elif len(all_rated_genres.mode()) > 1:
                second_most_frequent = all_rated_genres.mode().iloc[1]
                if pd.notna(second_most_frequent) and second_most_frequent.strip() != "" and second_most_frequent != 'Género Desconocido':
                    return second_most_frequent.strip()

        # Si aún no se encontró un género válido, elige uno aleatorio de los válidos
        valid_genres_in_list = [g for g in unique_genres_list if g and pd.notna(g) and g != 'Género Desconocido']
        if valid_genres_in_list:
            return np.random.choice(valid_genres_in_list)
        else:
            return "Preferencias desconocidas"
    else:
        # Si no hay calificaciones, elige un género aleatorio
        valid_genres_in_list = [g for g in unique_genres_list if g and pd.notna(g) and g != 'Género Desconocido']
        if valid_genres_in_list:
            return np.random.choice(valid_genres_in_list)
        else:
            return "Preferencias desconocidas"


# --- Configuración de Estilo Global (sin cambios) ---
background_color = "#f7eecd"
font_family = "'Roboto', 'Open Sans', 'Segoe UI', 'Arial', sans-serif"
base_container_style = f"background-color: {background_color}; font-family: {font_family}; padding: 15px; border-radius: 8px;"
text_style = f"color: #333; font-family: {font_family};"
light_text_style = f"color: #555; font-family: {font_family};"


print("--- Configuración de Nuevo Usuario Interactivo ---")

# --- 1. Crear un Nuevo User_ID y Perfil (Inicial - sin preferencia aún) ---
max_user_id = df_users['user_id'].max()
new_user_id = max_user_id + 1 # Usamos un ID muy alto para asegurar que no colisiona con usuarios existentes

new_user_data_initial = {
    'user_id': new_user_id,
    'avatar_url': f"https://api.dicebear.com/7.x/identicon/svg?seed={new_user_id}",
    'username': f'NuevoUsuario_{new_user_id}'
}

print(f"Configurando Nuevo Usuario '{new_user_data_initial['username']}' (ID: {new_user_id}).")
print("Por favor, selecciona sus características y califica algunos libros.")


# --- 2. Preparar Widgets Interactivos (Añadidos Desplegables de Perfil) ---

# Obtener opciones únicas de df_users, manejando NaNs
def get_unique_options(df, column):
    if column in df.columns:
        options = df[column].dropna().astype(str).unique().tolist()
        options.sort()
        options = [opt for opt in options if str(opt).strip() != '']
        return [f"--- Selecciona {column.capitalize()} ---"] + options
    else:
        print(f"Advertencia: Columna '{column}' no encontrada en df_users.")
        return [f"--- No hay datos de {column.capitalize()} ---"]

age_options = get_unique_options(df_users, 'age')
age_dropdown = widgets.Dropdown(options=age_options, description='Edad:')

gender_options = get_unique_options(df_users, 'gender')
gender_dropdown = widgets.Dropdown(options=gender_options, description='Género:')

education_options = get_unique_options(df_users, 'education')
education_dropdown = widgets.Dropdown(options=education_options, description='Educación:')

country_options = get_unique_options(df_users, 'country')
country_dropdown = widgets.Dropdown(options=country_options, description='País:')


# Desplegable de Géneros de Libros
genre_options = unique_genres_selector.copy()
genre_options.insert(0, "--- Selecciona un Género ---")

genre_dropdown = widgets.Dropdown(
    options=genre_options,
    description='Género:',
    disabled=False,
)

# Desplegable de Libros
book_dropdown = widgets.Dropdown(
    options=[("--- Selecciona un Género Primero ---", None)],
    description='Libro:',
    disabled=True,
)

# Desplegable de Calificación
rating_options = [(str(i), i) for i in range(1, 6)]
rating_dropdown = widgets.Dropdown(
    options=rating_options,
    description='Calificación:',
    disabled=False,
)

# Botón para Añadir Calificación
add_rating_button = widgets.Button(description="Añadir Calificación")
add_rating_button.disabled = True

# Área de salida para mostrar las calificaciones añadidas
output_ratings = widgets.Output()

# Lista para almacenar las calificaciones del nuevo usuario
new_user_ratings_collected = []


# --- 3. Lógica para Actualizar el Desplegable de Libros al Cambiar el Género (sin cambios) ---
def on_genre_change(change):
    selected_genre = change['new']
    book_dropdown.disabled = True
    add_rating_button.disabled = True
    book_dropdown.options = [("--- Cargando Libros ---", None)]
    book_dropdown.value = None

    if selected_genre == "--- Selecciona un Género ---":
        book_dropdown.options = [("--- Selecciona un Género Primero ---", None)]
    else:
        # Asegúrate de que df_combined_books['genre'] es string antes de usar .str.contains
        filtered_books = df_combined_books[
            df_combined_books['genre'].astype(str).str.contains(selected_genre, na=False, case=False)
        ]

        if not filtered_books.empty:
            # MODIFICADO: Intenta obtener 'authors' o 'author' para el desplegable de libros
            book_options_filtered = [(f"{row['title']} por {row.get('authors', row.get('author', 'Autor desconocido'))}", row['book_id'])
                                     for index, row in filtered_books.iterrows()]
            book_options_filtered.insert(0, ("--- Selecciona un Libro ---", None))

            book_dropdown.options = book_options_filtered
            book_dropdown.disabled = False
            if len(book_options_filtered) > 1:
                add_rating_button.disabled = False # Habilitar añadir calificación si hay libros para seleccionar

        else:
            book_dropdown.options = [("--- No se encontraron libros en este género ---", None)]


genre_dropdown.observe(on_genre_change, names='value')


# --- 4. Lógica para el Botón "Añadir Calificación" (sin cambios) ---
def on_add_rating_button_clicked(b):
    selected_book_id = book_dropdown.value
    selected_rating = rating_dropdown.value
    selected_book_title = book_dropdown.label

    if selected_book_id is not None and selected_rating is not None:
        already_rated = any(d['book_id'] == selected_book_id for d in new_user_ratings_collected)

        if not already_rated:
            new_user_ratings_collected.append({
                'user_id': new_user_id,
                'book_id': selected_book_id,
                'rating': selected_rating
            })
            with output_ratings:
                print(f"Añadida calificación: '{selected_book_title}' con {selected_rating} estrellas.")
        else:
            with output_ratings:
                print(f"Advertencia: Ya calificaste '{selected_book_title}'.")

    else:
        with output_ratings:
            print("Advertencia: Selecciona un libro y una calificación válidos.")

add_rating_button.on_click(on_add_rating_button_clicked)


# --- 5. Botón para Generar Recomendaciones y Lógica (ADAPTADA PARA EL ERROR 'get_params') ---
recommend_button = widgets.Button(description="Generar Recomendaciones")
output_recommendations = widgets.Output()
output_similar_user_books = widgets.Output() # Área de salida dedicada para mostrar libros de usuarios similares clicados

# --- Función para mostrar libros de un usuario similar clicado (¡MODIFICADA PARA MOSTRAR AUTOR Y ESTIMACIÓN!) ---
def display_similar_user_rated_books(similar_user_id, target_user_rated_books_set, recommended_book_ids_set,
                                     df_combined_books, user_item_matrix_param, output_widget, dynamic_svd_algo, new_user_id):
    """
    Muestra los libros calificados por un usuario similar, excluyendo los que el
    usuario objetivo ya calificó o los que ya fueron recomendados, e incluye la estimación para el nuevo usuario.
    """
    output_widget.clear_output()

    with output_widget:
        print(f"\nLibros calificados por Usuario Similar {similar_user_id} (excluyendo los tuyos y los ya recomendados):")

        if similar_user_id not in user_item_matrix_param.index:
            print(f"Info del usuario similar {similar_user_id} no encontrada en la matriz. Posiblemente no calificó nada en el dataset original.")
            return

        sim_user_ratings = user_item_matrix_param.loc[similar_user_id]
        sim_user_rated_book_ids = set(sim_user_ratings[sim_user_ratings > 0].index)

        books_to_show = sim_user_rated_book_ids - target_user_rated_books_set - recommended_book_ids_set

        if not books_to_show:
            print("Este usuario no calificó ningún libro que no tengas o que ya se te haya recomendado.")
            return
        
        books_details_to_show_with_estimates = []
        for book_id in books_to_show:
            details = df_combined_books[df_combined_books['book_id'] == int(book_id)]
            if not details.empty:
                book_detail = details.iloc[0]
                # Predecir la calificación para este libro para el nuevo usuario
                pred = dynamic_svd_algo.predict(new_user_id, book_id)
                estimated_score = pred.est
                books_details_to_show_with_estimates.append((book_id, estimated_score, book_detail))

        # Ordenar por la puntuación estimada para el nuevo usuario
        books_details_to_show_with_estimates.sort(key=lambda x: x[1], reverse=True)

        if not books_details_to_show_with_estimates: # Verificar si la lista está vacía después de procesar
            print("No se encontraron detalles para los libros calificados por este usuario o no hay recomendaciones para ellos.")
            return

        html_content = f"<div style='display: flex; flex-wrap: wrap; gap: 15px; {base_container_style}'>"

        for book_id, estimated_score, details in books_details_to_show_with_estimates: # Bucle sobre la nueva estructura
            book_title = details.get('title', "Título desconocido")
            image_url = details.get('image_url')
            # MODIFICADO: Intenta obtener 'authors' o 'author' para mayor robustez
            author = details.get('authors', details.get('author', "Autor desconocido")) 
            genre = details.get('genre', "Género desconocido")
            pages = details.get('pages', "Páginas desconocidas")

            pages_str = "Páginas desconocidas"
            try:
                if pd.notna(pages):
                    pages_str = f"{int(pages)} págs."
            except (ValueError, TypeError):
                pass

            html_content += f"""
            <div style="flex: 0 0 auto; width: 150px; text-align: center; overflow-wrap: break-word; border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color: #fff; font-family: {font_family};">
                <p style="font-size: 12px; font-weight: bold; margin-bottom: 5px; height: 3em; overflow: hidden; text-overflow: ellipsis; {text_style}">{book_title}</p>
                <img src="{image_url}" alt="Portada de {book_title}" width="100" style="display: block; margin: 0 auto; border: 1px solid #ccc; height: 150px; object-fit: cover;">
                <p style="font-size: 10px; margin-top: 5px; {light_text_style}>Autor: {author}</p>  <p style="font-size: 10px; {light_text_style}>Género: {genre}</p>  <p style="font-size: 10px; {light_text_style}>{pages_str}</p>
                <p style="font-size: 10px; font-weight: bold !important; color: green !important; margin-top: 5px; font-family: {font_family};">Estimación: {estimated_score:.2f}</p>  </div>
            """

        html_content += "</div>"
        display(HTML(html_content))


def on_recommend_button_clicked(b):
    output_recommendations.clear_output()
    output_similar_user_books.clear_output()

    # --- NEW: Capturar valores de los nuevos desplegables de perfil ---
    selected_age = age_dropdown.value if age_dropdown.value != '--- Selecciona Edad ---' else 'Desconocida'
    selected_gender = gender_dropdown.value if gender_dropdown.value != '--- Selecciona Género ---' else 'Desconocido'
    selected_education = education_dropdown.value if education_dropdown.value != '--- Selecciona Educación ---' else 'Desconocida'
    selected_country = country_dropdown.value if country_dropdown.value != '--- Selecciona País ---' else 'Desconocido'

    # Actualizar new_user_data_initial con los valores capturados
    new_user_data = new_user_data_initial.copy()
    new_user_data['age'] = selected_age
    new_user_data['gender'] = selected_gender
    new_user_data['education'] = selected_education
    new_user_data['country'] = selected_country


    if len(new_user_ratings_collected) < 3:
        with output_recommendations:
            print("Por favor, califica al menos 3 libros antes de generar recomendaciones.")
        return

    # Desactivar widgets durante la generación
    add_rating_button.disabled = True
    recommend_button.disabled = True
    genre_dropdown.disabled = True
    book_dropdown.disabled = True
    rating_dropdown.disabled = True

    with output_recommendations:
        print("Generando recomendaciones...")
        print("Re-entrenando SVD temporalmente con calificaciones actualizadas (esto puede tomar un momento)...")


    try:
        # Extraer listas de book_id y ratings de las calificaciones recogidas
        rated_book_ids = [d['book_id'] for d in new_user_ratings_collected]
        ratings = [d['rating'] for d in new_user_ratings_collected]

        # --- MODIFICACIÓN CLAVE AQUÍ: Re-entrenar SVD con las calificaciones interactivas ---
        # 1. Obtener los parámetros del modelo SVD optimizado del Bloque 5
        svd_tuned_params = {}
        try:
            svd_tuned_params['n_factors'] = getattr(best_svd_algo, 'n_factors', 100)
            svd_tuned_params['n_epochs'] = getattr(best_svd_algo, 'n_epochs', 20)
            svd_tuned_params['lr_all'] = getattr(best_svd_algo, 'lr_all', 0.005)
            svd_tuned_params['reg_all'] = getattr(best_svd_algo, 'reg_all', 0.02)
            print(f"Parámetros SVD obtenidos de best_svd_algo: {svd_tuned_params}")

        except AttributeError:
            print("Advertencia: No se pudieron extraer los parámetros SVD de 'best_svd_algo' directamente.")
            print("Usando valores de ejemplo (n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02).")
            print("Por favor, reemplaza estos con los parámetros óptimos de tu SVD (ej. de tu GridSearch).")
            svd_tuned_params = {
                'n_factors': 100,
                'n_epochs': 20,
                'lr_all': 0.005,
                'reg_all': 0.02
            }
        except Exception as e:
            print(f"Ocurrió un error inesperado al intentar obtener parámetros SVD: {e}")
            print("Usando valores de ejemplo.")
            svd_tuned_params = {
                'n_factors': 100,
                'n_epochs': 20,
                'lr_all': 0.005,
                'reg_all': 0.02
            }


        # 2. Crear una NUEVA instancia de SVD con esos parámetros
        dynamic_svd_algo = SVD(**svd_tuned_params)

        # 3. Combinar TODAS las calificaciones relevantes:
        all_current_ratings_df = pd.concat([df_ratings_modified, simulated_df, pd.DataFrame(new_user_ratings_collected)], ignore_index=True)

        # 4. Crear un trainset a partir de estos datos combinados
        reader = Reader(rating_scale=(1, 5))
        current_full_data = Dataset.load_from_df(all_current_ratings_df[['user_id', 'book_id', 'rating']], reader)
        current_trainset = current_full_data.build_full_trainset()

        # 5. Re-entrenar el modelo SVD TEMPORAL con el trainset completo y actualizado
        dynamic_svd_algo.fit(current_trainset)
        print("Modelo SVD temporal re-entrenado con éxito.")


        # --- CALCULAR READING PREFERENCE BASADA EN LAS CALIFICACIONES ---
        temp_new_user_ratings_df = pd.DataFrame(new_user_ratings_collected)
        calculated_reading_preference = assign_reading_preference(
            temp_new_user_ratings_df, df_combined_books, unique_genres
        )
        new_user_data['reading_preference'] = calculated_reading_preference


        # --- Encontrar Vecinos Similares (usando la lógica original de KNN) ---
        all_book_ids_in_matrix = user_item_matrix.columns
        num_books = len(all_book_ids_in_matrix)
        book_id_to_col_index = {book_id: i for i, book_id in enumerate(all_book_ids_in_matrix)}

        new_user_vector_dense = np.zeros(num_books)
        for rated_book_id, rating in zip(rated_book_ids, ratings):
            if rated_book_id in book_id_to_col_index:
                col_index = book_id_to_col_index[rated_book_id]
                new_user_vector_dense[col_index] = rating

        new_user_sparse_vector = csr_matrix(new_user_vector_dense.reshape(1, -1))

        n_neighbors_find = 15
        distances, indices = knn_best.kneighbors(new_user_sparse_vector, n_neighbors=n_neighbors_find)

        similarity_data = {'user_id': [], 'distance': []}
        for i in range(indices.shape[1]):
            user_idx = indices[0][i]
            dist = distances[0][i]
            original_user_id = user_item_matrix.index[user_idx]

            similarity_data['user_id'].append(original_user_id)
            similarity_data['distance'].append(dist)

        similarity_df_for_recommendation = pd.DataFrame(similarity_data)

        if knn_best.metric == 'cosine':
            similarity_df_for_recommendation['similarity'] = 1 - similarity_df_for_recommendation['distance']
        else:
            similarity_df_for_recommendation['similarity'] = 1 / (1 + similarity_df_for_recommendation['distance'])

        top_n_neighbors_for_recommendation = 10
        similarity_df_for_recommendation = similarity_df_for_recommendation.sort_values(by='similarity', ascending=False).head(top_n_neighbors_for_recommendation)


        if similarity_df_for_recommendation.empty:
            with output_recommendations:
                print("No se encontraron vecinos adecuados para generar recomendaciones.")


        # --- Lógica de SVD para Generar Recomendaciones de Libros (CON PRIORIZACIÓN DE GÉNERO) ---
        all_book_ids = set(df_combined_books['book_id'].unique())
        books_to_predict_for = list(all_book_ids - set(rated_book_ids))

        predictions = []
        for book_id in books_to_predict_for:
            pred = dynamic_svd_algo.predict(new_user_id, book_id)
            predictions.append((book_id, pred.est))

        predictions.sort(key=lambda x: x[1], reverse=True)

        top_books_svd_raw = predictions # Mantener todas las predicciones ordenadas

        final_recommended_books_details = []
        recommended_book_ids_set = set()

        num_svd_top_to_take = 5 # Tomar los 5 mejores de SVD puro inicialmente
        
        # 1. Tomar las N mejores recomendaciones puras de SVD
        for book_id, estimated_score in top_books_svd_raw[:num_svd_top_to_take]:
            book_details = df_combined_books[df_combined_books['book_id'] == int(book_id)]
            if not book_details.empty:
                details = book_details.iloc[0]
                final_recommended_books_details.append((book_id, estimated_score, details))
                recommended_book_ids_set.add(book_id)

        # 2. Añadir recomendaciones potenciadas por género (si hay una preferencia clara)
        if new_user_data.get('reading_preference') and new_user_data['reading_preference'] != 'Desconocida':
            pref_genre = new_user_data['reading_preference']
            genre_boosted_candidates = []

            for book_id, estimated_score in top_books_svd_raw: # Iterar a través de todas las predicciones SVD
                if book_id not in recommended_book_ids_set: # Solo considerar libros no añadidos ya
                    book_details = df_combined_books[df_combined_books['book_id'] == int(book_id)]
                    if not book_details.empty:
                        details = book_details.iloc[0]
                        # Verificar si el género preferido está en los géneros del libro (manejando múltiples géneros)
                        book_genres = str(details.get('genre', '')).split(', ')
                        if pref_genre in book_genres:
                            genre_boosted_candidates.append((book_id, estimated_score, details))
            
            # Ordenar los candidatos de género por puntuación
            genre_boosted_candidates.sort(key=lambda x: x[1], reverse=True)

            # Añadir hasta el total deseado (10 libros) de los candidatos potenciados por género
            for book_id, estimated_score, details in genre_boosted_candidates:
                if len(final_recommended_books_details) < 10:
                    final_recommended_books_details.append((book_id, estimated_score, details))
                    recommended_book_ids_set.add(book_id)
                else:
                    break
        
        # 3. Rellenar los slots restantes con las siguientes mejores predicciones SVD (si quedan slots)
        if len(final_recommended_books_details) < 10:
            for book_id, estimated_score in top_books_svd_raw:
                if book_id not in recommended_book_ids_set:
                    book_details = df_combined_books[df_combined_books['book_id'] == int(book_id)]
                    if not book_details.empty:
                        details = book_details.iloc[0]
                        final_recommended_books_details.append((book_id, estimated_score, details))
                        recommended_book_ids_set.add(book_id)
                        if len(final_recommended_books_details) >= 10:
                            break

        # Ordenar la lista final por puntuación estimada para asegurar que los 10 primeros son los mejores
        final_recommended_books_details.sort(key=lambda x: x[1], reverse=True)
        final_recommended_books_details = final_recommended_books_details[:10] # Asegurar que solo hay 10


        # --- Mostrar Resultados ---
        with output_recommendations:
            # Mostrar perfil del nuevo usuario
            user_html = f"""
            <div style="{base_container_style} margin-bottom: 20px;">
                <div style="display: flex; align-items: center; border-bottom: 1px solid #eee; padding-bottom: 15px;">
                    <div style="margin-right: 20px;">
                        <img src="{new_user_data['avatar_url']}" alt="Avatar de {new_user_data['username']}" width="100" style="border-radius: 50%; border: 2px solid #ccc; object-fit: cover;">
                    </div>
                    <div>
                        <h3 style="margin: 0 0 5px 0; {text_style}">Perfil del Usuario Seleccionado: <span style="color: #007bff; font-weight: bold;">{new_user_data['username']}</span> (ID: {new_user_id})</h3>
                        <p style="margin: 0; font-size: 14px; {light_text_style}">Edad: {new_user_data.get('age', 'Desconocida')}</p>
                        <p style="margin: 0; font-size: 14px; {light_text_style}>Género: {new_user_data.get('gender', 'Desconocido')}</p>
                        <p style="margin: 0; font-size: 14px; {light_text_style}>Educación: {new_user_data.get('education', 'Desconocida')}</p>
                        <p style="margin: 0; font-size: 14px; {light_text_style}>País: {new_user_data.get('country', 'Desconocido')}</p>
                        <p style="margin: 0; font-size: 14px; {light_text_style}>Preferencia de lectura: {new_user_data.get('reading_preference', 'Desconocida')}</p>
                    </div>
                </div>
            </div>
            """
            display(HTML(user_html))

            num_similar_users_to_show_in_display = 4

            # Asegúrate de que similarity_df_for_recommendation esté definida aquí
            top_similar_users_for_display = similarity_df_for_recommendation.head(num_similar_users_to_show_in_display)
            top_similar_user_ids_for_display = top_similar_users_for_display['user_id'].tolist()

            similar_user_widgets_list = []

            top_similar_users_info_display = df_users[df_users['user_id'].isin(top_similar_user_ids_for_display)].set_index('user_id')


            if not top_similar_users_info_display.empty:
                similar_users_title_html = f"<div style='{base_container_style} margin-bottom: 10px;'>"
                similar_users_title_html += f"<h3 style='margin-top: 0; margin-bottom: 5px; {text_style}'>Usuarios Similares (Haz clic para ver sus libros):</h3>"
                similar_users_title_html += "</div>"
                display(HTML(similar_users_title_html))

                similar_users_hbox_container = widgets.HBox([])
                similar_users_hbox_container.layout.justify_content = 'flex-start'
                similar_users_hbox_container.layout.margin = '0 0 10px 0'


                for sim_user_id in top_similar_users_for_display['user_id']:
                    sim_user_info = top_similar_users_info_display.loc[sim_user_id] if sim_user_id in top_similar_users_info_display.index else None

                    if sim_user_info is not None:
                        sim_user_avatar_url = sim_user_info.get('avatar_url', f"https://api.dicebear.com/7.x/identicon/svg?seed={sim_user_id}")
                        sim_user_username = sim_user_info.get('username', f'Usuario {sim_user_id}')

                        avatar_html_value = f"""
                        <div style="text-align: center; width: 80px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap; font-family: {font_family};">
                            <img src="{sim_user_avatar_url}" alt="Portada de {sim_user_username}" width="50" height="50" style="border-radius: 50%; border: 1px solid #ccc; object-fit: cover; display: block; margin: 0 auto;"><br>
                            <span style="font-size: 10px; {light_text_style}" title="{sim_user_username}">{sim_user_username}</span>
                        </div>
                        """
                        avatar_widget_html = widgets.HTML(value=avatar_html_value)

                        action_button = widgets.Button(
                            description='',
                            layout=widgets.Layout(width='80px', height='80px', margin='-80px 0 0 0'),
                            style=widgets.ButtonStyle(
                                button_color='transparent',
                                border='1px solid transparent',
                                cursor='pointer'
                            )
                        )
                        action_button._similar_user_id = sim_user_id

                        # ¡MODIFICADO!: Pasa dynamic_svd_algo y new_user_id a la función de visualización
                        action_button.on_click(lambda b: display_similar_user_rated_books(
                            b._similar_user_id,
                            set(rated_book_ids),
                            recommended_book_ids_set,
                            df_combined_books,
                            user_item_matrix,
                            output_similar_user_books,
                            dynamic_svd_algo, # NUEVO PARÁMETRO
                            new_user_id # NUEVO PARÁMETRO
                        ))

                        user_widget_vbox = widgets.VBox([avatar_widget_html, action_button], layout=widgets.Layout(align_items='center', margin='0', padding='0', width='90px'))

                        similar_user_widgets_list.append(user_widget_vbox)

                if similar_user_widgets_list:
                    similar_users_hbox_container.children = similar_user_widgets_list
                    display(similar_users_hbox_container)
                else:
                    print("No se pudieron crear widgets para usuarios similares.")
            else:
                print("\nNo se encontraron usuarios similares para mostrar.")

            print(f"\n--- Top 10 Libros Recomendados para {new_user_data['username']} (ID: {new_user_id}): ---")

            # La nota de la estimación ha sido eliminada de aquí.

            if not final_recommended_books_details:
                print("No se encontraron recomendaciones de libros.")
            else:
                html_content = f"<div style='display: flex; flex-wrap: wrap; gap: 15px; {base_container_style}'>"

                for book_id, estimated_score, details in final_recommended_books_details:
                    book_title = details.get('title', "Título desconocido")
                    image_url = details.get('image_url')
                    # MODIFICADO: Intenta obtener 'authors' o 'author' para mayor robustez
                    author = details.get('authors', details.get('author', "Autor desconocido")) 
                    genre = details.get('genre', "Género desconocido")
                    pages = details.get('pages', "Páginas desconocidas")

                    pages_str = "Páginas desconocidas"
                    try:
                        if pd.notna(pages):
                            pages_str = f"{int(pages)} págs."
                    except (ValueError, TypeError):
                        pass

                    html_content += f"""
                    <div style="flex: 0 0 auto; width: 150px; text-align: center; overflow-wrap: break-word; border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color: #fff; font-family: {font_family};">
                        <p style="font-size: 12px; font-weight: bold; margin-bottom: 5px; height: 3em; overflow: hidden; text-overflow: ellipsis; {text_style}">{book_title}</p>
                        <img src="{image_url}" alt="Portada de {book_title}" width="100" style="display: block; margin: 0 auto; border: 1px solid #ccc; height: 150px; object-fit: cover;">
                        <p style="font-size: 10px; margin-top: 5px; {light_text_style}>Autor: {author}</p> <p style="font-size: 10px; {light_text_style}>Género: {genre}</p>
                        <p style="font-size: 10px; font-weight: bold !important; color: green !important; margin-top: 5px; font-family: {font_family};">Estimación: {estimated_score:.2f}</p> </div>
                    """

                html_content += "</div>"
                display(HTML(html_content))


    except Exception as e:
        with output_recommendations:
            print(f"Ocurrió un error al generar recomendaciones: {e}")

    finally:
        # Reactivar widgets al finalizar
        add_rating_button.disabled = False
        recommend_button.disabled = False
        genre_dropdown.disabled = False
        if genre_dropdown.value != "--- Selecciona un Género ---":
            book_dropdown.disabled = False
            if book_dropdown.value is not None:
                add_rating_button.disabled = False
        rating_dropdown.disabled = False


# Vincular la lógica al evento click del botón de recomendación
recommend_button.on_click(on_recommend_button_clicked)


# --- 6. Mostrar los Widgets Interactivos ---
print("\nSelecciona las características del usuario, un género y califica libros:")
interactive_widgets = widgets.VBox([
    age_dropdown,
    gender_dropdown,
    education_dropdown,
    country_dropdown,

    genre_dropdown,
    book_dropdown,
    rating_dropdown,
    add_rating_button,
    output_ratings,
    recommend_button,
    output_recommendations,
    output_similar_user_books
])

display(interactive_widgets)

print("\n--- Bloque 7 (Interfaz) Completado ---")

------------------

### **5.4.2 Explanation**

### **Explicación del Resultado de la Recomendación**  
  
El resultado de la recomendación para el usuario X significa que los libros mostrados son aquellos que:

1. Fueron calificados (presumiblemente bien) por otros usuarios que tienen gustos similares a los del usuario X (basado en sus patrones de calificación histórica).
2. El usuario X aún no ha calificado (o no aparece como calificado en la user_item_matrix).
3. Tienen la mayor "relevancia" calculada (la suma de las similitudes de los vecinos que calificaron ese libro) entre los libros que cumplen los criterios anteriores.

Esencialmente, el sistema dice: "Basándonos en lo que te gusta (tus calificaciones), hemos encontrado a otras personas a las que les gustan cosas similares. Estos son algunos libros que a ellos les gustaron y que tú aún no has leído."

### **¿Por qué el número de recomendaciones varía?**  
  
1. *Número y Calidad de Vecinos Similares*: Si un usuario tiene pocos vecinos realmente similares (es decir, usuarios con patrones de calificación muy parecidos), hay menos fuentes de donde sacar recomendaciones.
  
2. *Superposición de Libros Calificados:*: El algoritmo recomienda libros que los usuarios similares han calificado pero que el usuario objetivo aún no ha calificado.
Si los vecinos más cercanos de un usuario han calificado predominantemente los mismos libros que el usuario objetivo ya ha calificado, habrá muy pocos libros nuevos para recomendar después de aplicar el filtro.
  
3. *Sparsidad de los Datos (Pocos Ratings):*: Si tanto el usuario objetivo como sus vecinos similares han calificado muy pocos libros en total, la cantidad de libros "nuevos" (no calificados por el objetivo pero calificados por vecinos) será baja.

<div>
<img src="idea_user-based_knn.png" width="500"/>
</div>

# Resumen del Flujo de Trabajo por Bloque

---

### **Bloque 1: Carga de Datos Inicial**
* **¿Qué hace?** Carga tus datasets principales (como `users.csv`, `books.csv`, `ratings.csv`) en **DataFrames de Pandas**. Es donde todo comienza en tu proyecto.

---

### **Bloque 2: Preprocesamiento y Limpieza de Datos**
* **¿Qué hace?** Prepara tus datos para el análisis. Aquí realizas tareas como:
    * Gestionar valores nulos o inconsistentes.
    * Convertir tipos de datos si es necesario.
    * **Combinar DataFrames** (ej. ratings con libros y usuarios) para una vista más completa.
    * Filtrar datos, si es necesario, o crear nuevas columnas (características).

---

### **Bloque 3: Análisis Exploratorio de Datos (EDA)**
* **¿Qué hace?** Te ayuda a entender a fondo tus datos. Esto implica:
    * Calcular **estadísticas descriptivas** (promedios, medianas, etc.).
    * Crear **visualizaciones** (gráficos, histogramas) para descubrir patrones, distribuciones y posibles problemas en los datos.
    * Obtener insights valiosos sobre el comportamiento de los usuarios, las características de los libros y los patrones de calificación.

---

### **Bloque 4: Configuración del Entorno de Machine Learning**
* **¿Qué hace?** Prepara tus datos para que los modelos de recomendación puedan usarlos:
    * Define el **`Reader`** para la librería `Surprise`, especificando la escala de las calificaciones (ej. de 1 a 5).
    * Carga tus datos preprocesados en el formato específico de `Surprise` (`Dataset`).
    * **Divide el dataset** en conjuntos de entrenamiento (`trainset`) y prueba (`testset`), fundamental para evaluar el rendimiento del modelo.

---

### **Bloque 5: Entrenamiento y Optimización del Modelo SVD**
* **¿Qué hace?** Es el corazón de tu sistema de recomendación:
    * Inicializa y **entrena el algoritmo SVD** (o el algoritmo de recomendación que hayas elegido) usando el conjunto de entrenamiento.
    * Realiza **validación cruzada** y, posiblemente, optimización de hiperparámetros (ej. con `GridSearchCV`) para encontrar la configuración ideal del modelo.
    * Evalúa el rendimiento del modelo usando métricas como RMSE o MAE.
    * **Guarda el "mejor" modelo SVD optimizado** para que puedas usarlo más tarde sin necesidad de re-entrenar desde cero.

---

### **Bloque 6: Preparación del Modelo para un Nuevo Usuario**
* **¿Qué hace?** Prepara tu sistema para interactuar con un usuario que no estaba en tus datos originales:
    * Define un **`new_user_id`** (un ID único para este nuevo usuario).
    * **Simula unas pocas calificaciones iniciales** para este `new_user_id` con libros que sabes que existen. Esto es vital para que el modelo SVD pueda aprender sus factores latentes durante el re-entrenamiento.
    * Combina estas calificaciones simuladas con tu dataset de ratings existente.
    * **Re-entrena tu `best_svd_algo`** con este dataset actualizado, de forma que el modelo ya "conozca" al nuevo usuario y pueda hacer predicciones significativas para él.

---

### **Bloque 7: Interfaz Interactiva y Recomendaciones para el Nuevo Usuario**
* **¿Qué hace?** Proporciona la experiencia de usuario final para obtener recomendaciones:
    * Recopila **calificaciones adicionales** del `new_user_id` de forma interactiva.
    * Usa estas calificaciones para entender mejor las preferencias del usuario (ej. sus géneros favoritos).
    * Filtra los libros que el usuario ya ha calificado para no recomendarlos de nuevo.
    * Genera y presenta **recomendaciones personalizadas de libros no calificados** utilizando el `best_svd_algo` (que ya está listo para este `new_user_id` gracias al Bloque 6).

--------------

# **GUARDAR INFORMACIÓN PARA EL STREAMLIT**

Content-Based

In [ ]:
df_combined_books.to_parquet('df_combined_books_final.parquet', index=False)
print("df_combined_books_final.parquet guardado con éxito.")

In [ ]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np # Necesario para float32
from scipy.sparse import csr_matrix, save_npz # <-- ¡NUEVAS IMPORTACIONES!

# --- Asegúrate de que df_combined_books está cargado y es el DataFrame final ---
# Si acabas de guardar 'df_combined_books_final.parquet', cárgalo de nuevo aquí para asegurarte:
df_combined_books = pd.read_parquet('df_combined_books_final.parquet')


# --- Preparación de 'features' (exactamente como lo tenías) ---
# Maneja NaNs si es necesario antes de convertir a str y combinar
# Asegúrate de que las columnas existan y manejen NaN adecuadamente antes de .astype(str)
for col in ['title', 'authors', 'average_rating', 'genre', 'pages']:
    if col not in df_combined_books.columns:
        print(f"Advertencia: La columna '{col}' no se encontró en df_combined_books.")
        df_combined_books[col] = '' # Asigna una cadena vacía si falta para evitar errores

features['title'] = features['title'].str.lower()
features['authors'] = features['authors'].str.lower()
features['average_rating'] = features['average_rating'].str.lower()
features['genre'] = features['genre'].str.lower()
features['pages'] = features['pages'].str.lower()

features['combined_features'] = features['title'] + ' ' + \
                                features['authors'] + ' ' + \
                                features['average_rating'] + ' ' + \
                                features['genre'] + ' ' + \
                                features['pages']

# --- TF-IDF y similitud de coseno ---
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(features['combined_features'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# --- AQUI ESTÁN LOS CAMBIOS IMPORTANTES PARA COMPRIMIR ---
print(f"Tamaño inicial de cosine_sim (float64): {cosine_sim.nbytes / (1024*1024):.2f} MB")

# 1. Convertir a float32 para reducir a la mitad el tamaño inicial
cosine_sim = cosine_sim.astype(np.float32)
print(f"Tamaño después de float32: {cosine_sim.nbytes / (1024*1024):.2f} MB")

# 2. Convierte a matriz dispersa (csr_matrix)
# APLICA UN UMBRAL ANTES DE CONVERTIR A DISPERSA
# Esto convertirá valores muy pequeños (casi cero) a cero, haciendo la matriz más dispersa.
# EL VALOR 1e-6 ES UN EJEMPLO. PODRÍAS AJUSTARLO (ej: 0.01, 0.05, 0.1) PARA CONTROLAR CUÁNTO ELIMINAS.
# SI LO AJUSTAS DEMASIADO ALTO, REDUCIRÁS LA CALIDAD DE LAS RECOMENDACIONES.
threshold = 0.05 #1e-6 # O un valor mayor como 0.01 o 0.05 si necesitas más esparcimiento y puedes sacrificar precisión.
cosine_sim[cosine_sim < threshold] = 0 # Establece a cero los valores por debajo del umbral

cosine_sim_sparse = csr_matrix(cosine_sim) # Ahora, esta matriz será más dispersa
print(f"Número de elementos no-cero en la matriz dispersa (después de umbral): {cosine_sim_sparse.nnz}")

# --- Guarda los objetos calculados ---
# Guardamos tfidf_vectorizer.pkl como siempre
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

# ¡Guardamos la matriz de similitud como .npz y en formato disperso!
# Asegúrate de que el nombre del archivo es 'cosine_sim_matrix.npz'
save_npz('cosine_sim_matrix.npz', cosine_sim_sparse) # <-- ¡CAMBIO AQUÍ!

print("tfidf_vectorizer.pkl guardado con éxito.")
print("cosine_sim_matrix.npz (matriz dispersa) guardado con éxito.")

# Opcional: También guarda el DataFrame 'features' si lo usas para indexación más tarde
# features.to_parquet('features_processed.parquet', index=False)